In [112]:
# import libraries 
import pandas as pd
import numpy as np
import subprocess
import re

In [113]:
#read the merged FF with base before and after  file as tsv 
# old ver: "/private5/Projects/dadush/clinvar_base_editing/Erez_15_03_23/Erez_merged_with_DNA_seq.tsv" 
clinvar_path = "/private5/Projects/dadush/clinvar_base_editing/Erez_15_03_23/Erez_without_merged_ver_1_with_DNA_seq.tsv"
clinvar_df = pd.read_csv(clinvar_path, sep='\t')
# change name column of dna seq
clinvar_df = clinvar_df.rename({'nAn':'dna_seq'}, axis=1)
pd.set_option('display.max_columns', None)
clinvar_df


/tmp/ipykernel_4302/3271661992.py:4: DtypeWarning: Columns (41) have mixed types. Specify dtype option on import or set low_memory=False.
  clinvar_df = pd.read_csv(clinvar_path, sep='\t')


,chrom,chromStart,chromEnd,name,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,dna_seq
0,chr1,943994,943995,C>T,1,+,943994,943995,"210,0,0",1,1,0,950448|NM_001385641.1(SAMD11):c.2377C>T (p.Arg...,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,148398|SAMD11,nonsense,nAn,nAn,RCV001222161,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.943995C>T"", ""-""], [""c.2380C>T"", ""p.Arg794...",nAn,1,"Jul 13,2020",nAn,nAn,NM_001385641.1(SAMD11):c.2377C>T (p.Arg793Ter)...,P,GERM,SUBST,1,1,950448,nAn,1,943995.0,950448.0,C,T,.,.,ALLELEID=929884;CLNDISDB=MedGen:CN517202;CLNDN...,chr1,Pathogenic,nAn,nAn,G,G,.,NM_001385641.1,CAACCCTGCGGGCCCCGGAGCGAGAACTCGGCACAGGAGAG
1,chr1,964511,964512,C>A,0,+,964511,964512,"210,0,0",1,1,0,916564|NM_198317.3(KLHL17):c.1682C>A (p.Ala561...,Likely pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,339451|KLHL17,missense variant,nAn,nAn,RCV001172295,N,Tracheoesophageal fistula,"Human Phenotype Ontology:HP:0002575, MONDO:MON...",de novo,GRCh38,1p36.33,"[[""g.964512C>A"", ""-""], [""g.8926C>A"", ""-""], [""c...",nAn,1,"Jul 01,2019",nAn,nAn,"NM_198317.3(KLHL17):c.1682C>A (p.Ala561Glu), T...",LP,GERM,SUBST,1,0,916564,nAn,1,964512.0,916564.0,C,A,.,.,ALLELEID=904889;CLNDISDB=Human_Phenotype_Ontol...,chr1,Likely_pathogenic,nAn,nAn,G,G,.,NM_198317.3,TGGAGCCTGGGAAAGCGTGGCGCCCATGAATATCCGCAGGT
2,chr1,1013982,1013983,G>A,1,+,1013982,1013983,"210,0,0",1,1,0,1028857|NM_005101.4(ISG15):c.4-1G>A,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,splice acceptor variant,nAn,nAn,RCV001330014,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""-""], [""g.1013983G>A"", ""-""]...",nAn,1,"Feb 21,2020",nAn,nAn,"NM_005101.4(ISG15):c.4-1G>A, Type: single nucl...",P,UNK,SUBST,1,1,1028857,nAn,1,1013983.0,1028857.0,G,A,.,.,"ALLELEID=1015460;CLNDISDB=MONDO:MONDO:0014502,...",chr1,Pathogenic,nAn,nAn,A,G,.,NM_005101.4,GAACCTGTGTGACGCCTGCAGGGCTGGGACCTGACGGTGAA
3,chr1,1014142,1014143,C>T,0,+,1014142,1014143,"210,0,0",1,1,0,183381|NM_005101.4(ISG15):c.163C>T (p.Gln55Ter),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,nonsense,786201005.0,nAn,RCV000162196,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""c.163C>T"", ""p.Gln55Ter""], [""g.18006C>T"", ""-...",nAn,1,"Jan 01,2015",nAn,"ClinGen:CA186084, <a target=_blank href='http:...","NM_005101.4(ISG15):c.163C>T (p.Gln55Ter), Type...",P,GERM,SUBST,1,0,183381,nAn,1,1014143.0,183381.0,C,T,.,.,"ALLELEID=181485;CLNDISDB=MONDO:MONDO:0014502,M...",chr1,Pathogenic,nAn,nAn,G,A,.,NM_005101.4,ACCCGAGCGGTGTGGCGCTGCAGGACAGGGTCCCCCTTGCC
4,chr1,1014227,1014228,G>A,1,+,1014227,1014228,"137,121,212",1,1,0,402986|NM_005101.4(ISG15):c.248G>A (p.Ser83Asn),Conflicting interpretations of pathogenicity,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,missense variant,1921.0,nAn,RCV000455759|RCV001519553|RCV001727727,N,not specified|Immunodeficiency 38 with basal g...,"MedGen:CN169374|MONDO:MONDO:0014502, <a target...","de novo,germline",GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""p.Ser83Asn""], [""c.248G>A"",...",nAn,4,"Sep 22,2021",nAn,ClinGen:CA507658,"NM_005101.4(ISG15):c.248G>A (p.Ser83Asn), Type...",CF,GERM,SUBST,1,1,402986,nAn

check every column 

In [114]:
# Distribution of the mutations according to Strand
clinvar_df.groupby(['Strand'])['Strand'].count()
# visual test
clinvar_df[clinvar_df['Strand']=='+']

,chrom,chromStart,chromEnd,name,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,dna_seq
0,chr1,943994,943995,C>T,1,+,943994,943995,"210,0,0",1,1,0,950448|NM_001385641.1(SAMD11):c.2377C>T (p.Arg...,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,148398|SAMD11,nonsense,nAn,nAn,RCV001222161,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.943995C>T"", ""-""], [""c.2380C>T"", ""p.Arg794...",nAn,1,"Jul 13,2020",nAn,nAn,NM_001385641.1(SAMD11):c.2377C>T (p.Arg793Ter)...,P,GERM,SUBST,1,1,950448,nAn,1,943995.0,950448.0,C,T,.,.,ALLELEID=929884;CLNDISDB=MedGen:CN517202;CLNDN...,chr1,Pathogenic,nAn,nAn,G,G,.,NM_001385641.1,CAACCCTGCGGGCCCCGGAGCGAGAACTCGGCACAGGAGAG
1,chr1,964511,964512,C>A,0,+,964511,964512,"210,0,0",1,1,0,916564|NM_198317.3(KLHL17):c.1682C>A (p.Ala561...,Likely pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,339451|KLHL17,missense variant,nAn,nAn,RCV001172295,N,Tracheoesophageal fistula,"Human Phenotype Ontology:HP:0002575, MONDO:MON...",de novo,GRCh38,1p36.33,"[[""g.964512C>A"", ""-""], [""g.8926C>A"", ""-""], [""c...",nAn,1,"Jul 01,2019",nAn,nAn,"NM_198317.3(KLHL17):c.1682C>A (p.Ala561Glu), T...",LP,GERM,SUBST,1,0,916564,nAn,1,964512.0,916564.0,C,A,.,.,ALLELEID=904889;CLNDISDB=Human_Phenotype_Ontol...,chr1,Likely_pathogenic,nAn,nAn,G,G,.,NM_198317.3,TGGAGCCTGGGAAAGCGTGGCGCCCATGAATATCCGCAGGT
2,chr1,1013982,1013983,G>A,1,+,1013982,1013983,"210,0,0",1,1,0,1028857|NM_005101.4(ISG15):c.4-1G>A,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,splice acceptor variant,nAn,nAn,RCV001330014,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""-""], [""g.1013983G>A"", ""-""]...",nAn,1,"Feb 21,2020",nAn,nAn,"NM_005101.4(ISG15):c.4-1G>A, Type: single nucl...",P,UNK,SUBST,1,1,1028857,nAn,1,1013983.0,1028857.0,G,A,.,.,"ALLELEID=1015460;CLNDISDB=MONDO:MONDO:0014502,...",chr1,Pathogenic,nAn,nAn,A,G,.,NM_005101.4,GAACCTGTGTGACGCCTGCAGGGCTGGGACCTGACGGTGAA
3,chr1,1014142,1014143,C>T,0,+,1014142,1014143,"210,0,0",1,1,0,183381|NM_005101.4(ISG15):c.163C>T (p.Gln55Ter),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,nonsense,786201005.0,nAn,RCV000162196,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""c.163C>T"", ""p.Gln55Ter""], [""g.18006C>T"", ""-...",nAn,1,"Jan 01,2015",nAn,"ClinGen:CA186084, <a target=_blank href='http:...","NM_005101.4(ISG15):c.163C>T (p.Gln55Ter), Type...",P,GERM,SUBST,1,0,183381,nAn,1,1014143.0,183381.0,C,T,.,.,"ALLELEID=181485;CLNDISDB=MONDO:MONDO:0014502,M...",chr1,Pathogenic,nAn,nAn,G,A,.,NM_005101.4,ACCCGAGCGGTGTGGCGCTGCAGGACAGGGTCCCCCTTGCC
4,chr1,1014227,1014228,G>A,1,+,1014227,1014228,"137,121,212",1,1,0,402986|NM_005101.4(ISG15):c.248G>A (p.Ser83Asn),Conflicting interpretations of pathogenicity,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,missense variant,1921.0,nAn,RCV000455759|RCV001519553|RCV001727727,N,not specified|Immunodeficiency 38 with basal g...,"MedGen:CN169374|MONDO:MONDO:0014502, <a target...","de novo,germline",GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""p.Ser83Asn""], [""c.248G>A"",...",nAn,4,"Sep 22,2021",nAn,ClinGen:CA507658,"NM_005101.4(ISG15):c.248G>A (p.Ser83Asn), Type...",CF,GERM,SUBST,1,1,402986,nAn

In [115]:
# Distribution of the mutations according to chrom
clinvar_df.groupby(['chrom'])['chrom'].count()
# Checking if there is a nan value
clinvar_df[clinvar_df['chrom'].isna()]

,chrom,chromStart,chromEnd,name,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,dna_seq


In [116]:
# Distribution of the mutations according to molConseq
clinvar_df.groupby(['molConseq'])['molConseq'].count()

molConseq
3 prime UTR variant                 50
5 prime UTR variant                117
initiatior codon variant           939
intron variant                    2441
missense variant                 47748
non-coding transcript variant      238
nonsense                         30237
splice acceptor variant           7224
splice donor variant              8803
stop lost                          155
synonymous variant                 561
Name: molConseq, dtype: int64

In [117]:
# Checking if there is a nan value
clinvar_df[clinvar_df['chromStart'].isna()]
# Checking that all values are numbers
clinvar_df[clinvar_df['chromStart'].astype(str).str.isdigit()]

,chrom,chromStart,chromEnd,name,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,dna_seq
0,chr1,943994,943995,C>T,1,+,943994,943995,"210,0,0",1,1,0,950448|NM_001385641.1(SAMD11):c.2377C>T (p.Arg...,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,148398|SAMD11,nonsense,nAn,nAn,RCV001222161,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.943995C>T"", ""-""], [""c.2380C>T"", ""p.Arg794...",nAn,1,"Jul 13,2020",nAn,nAn,NM_001385641.1(SAMD11):c.2377C>T (p.Arg793Ter)...,P,GERM,SUBST,1,1,950448,nAn,1,943995.0,950448.0,C,T,.,.,ALLELEID=929884;CLNDISDB=MedGen:CN517202;CLNDN...,chr1,Pathogenic,nAn,nAn,G,G,.,NM_001385641.1,CAACCCTGCGGGCCCCGGAGCGAGAACTCGGCACAGGAGAG
1,chr1,964511,964512,C>A,0,+,964511,964512,"210,0,0",1,1,0,916564|NM_198317.3(KLHL17):c.1682C>A (p.Ala561...,Likely pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,339451|KLHL17,missense variant,nAn,nAn,RCV001172295,N,Tracheoesophageal fistula,"Human Phenotype Ontology:HP:0002575, MONDO:MON...",de novo,GRCh38,1p36.33,"[[""g.964512C>A"", ""-""], [""g.8926C>A"", ""-""], [""c...",nAn,1,"Jul 01,2019",nAn,nAn,"NM_198317.3(KLHL17):c.1682C>A (p.Ala561Glu), T...",LP,GERM,SUBST,1,0,916564,nAn,1,964512.0,916564.0,C,A,.,.,ALLELEID=904889;CLNDISDB=Human_Phenotype_Ontol...,chr1,Likely_pathogenic,nAn,nAn,G,G,.,NM_198317.3,TGGAGCCTGGGAAAGCGTGGCGCCCATGAATATCCGCAGGT
2,chr1,1013982,1013983,G>A,1,+,1013982,1013983,"210,0,0",1,1,0,1028857|NM_005101.4(ISG15):c.4-1G>A,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,splice acceptor variant,nAn,nAn,RCV001330014,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""-""], [""g.1013983G>A"", ""-""]...",nAn,1,"Feb 21,2020",nAn,nAn,"NM_005101.4(ISG15):c.4-1G>A, Type: single nucl...",P,UNK,SUBST,1,1,1028857,nAn,1,1013983.0,1028857.0,G,A,.,.,"ALLELEID=1015460;CLNDISDB=MONDO:MONDO:0014502,...",chr1,Pathogenic,nAn,nAn,A,G,.,NM_005101.4,GAACCTGTGTGACGCCTGCAGGGCTGGGACCTGACGGTGAA
3,chr1,1014142,1014143,C>T,0,+,1014142,1014143,"210,0,0",1,1,0,183381|NM_005101.4(ISG15):c.163C>T (p.Gln55Ter),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,nonsense,786201005.0,nAn,RCV000162196,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""c.163C>T"", ""p.Gln55Ter""], [""g.18006C>T"", ""-...",nAn,1,"Jan 01,2015",nAn,"ClinGen:CA186084, <a target=_blank href='http:...","NM_005101.4(ISG15):c.163C>T (p.Gln55Ter), Type...",P,GERM,SUBST,1,0,183381,nAn,1,1014143.0,183381.0,C,T,.,.,"ALLELEID=181485;CLNDISDB=MONDO:MONDO:0014502,M...",chr1,Pathogenic,nAn,nAn,G,A,.,NM_005101.4,ACCCGAGCGGTGTGGCGCTGCAGGACAGGGTCCCCCTTGCC
4,chr1,1014227,1014228,G>A,1,+,1014227,1014228,"137,121,212",1,1,0,402986|NM_005101.4(ISG15):c.248G>A (p.Ser83Asn),Conflicting interpretations of pathogenicity,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,missense variant,1921.0,nAn,RCV000455759|RCV001519553|RCV001727727,N,not specified|Immunodeficiency 38 with basal g...,"MedGen:CN169374|MONDO:MONDO:0014502, <a target...","de novo,germline",GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""p.Ser83Asn""], [""c.248G>A"",...",nAn,4,"Sep 22,2021",nAn,ClinGen:CA507658,"NM_005101.4(ISG15):c.248G>A (p.Ser83Asn), Type...",CF,GERM,SUBST,1,1,402986,nAn

In [118]:
# Checking if there is a nan value
clinvar_df[clinvar_df['chromEnd'].isna()]
# Checking that all values are numbers
clinvar_df[clinvar_df['chromEnd'].astype(str).str.isdigit()]

,chrom,chromStart,chromEnd,name,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,dna_seq
0,chr1,943994,943995,C>T,1,+,943994,943995,"210,0,0",1,1,0,950448|NM_001385641.1(SAMD11):c.2377C>T (p.Arg...,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,148398|SAMD11,nonsense,nAn,nAn,RCV001222161,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.943995C>T"", ""-""], [""c.2380C>T"", ""p.Arg794...",nAn,1,"Jul 13,2020",nAn,nAn,NM_001385641.1(SAMD11):c.2377C>T (p.Arg793Ter)...,P,GERM,SUBST,1,1,950448,nAn,1,943995.0,950448.0,C,T,.,.,ALLELEID=929884;CLNDISDB=MedGen:CN517202;CLNDN...,chr1,Pathogenic,nAn,nAn,G,G,.,NM_001385641.1,CAACCCTGCGGGCCCCGGAGCGAGAACTCGGCACAGGAGAG
1,chr1,964511,964512,C>A,0,+,964511,964512,"210,0,0",1,1,0,916564|NM_198317.3(KLHL17):c.1682C>A (p.Ala561...,Likely pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,339451|KLHL17,missense variant,nAn,nAn,RCV001172295,N,Tracheoesophageal fistula,"Human Phenotype Ontology:HP:0002575, MONDO:MON...",de novo,GRCh38,1p36.33,"[[""g.964512C>A"", ""-""], [""g.8926C>A"", ""-""], [""c...",nAn,1,"Jul 01,2019",nAn,nAn,"NM_198317.3(KLHL17):c.1682C>A (p.Ala561Glu), T...",LP,GERM,SUBST,1,0,916564,nAn,1,964512.0,916564.0,C,A,.,.,ALLELEID=904889;CLNDISDB=Human_Phenotype_Ontol...,chr1,Likely_pathogenic,nAn,nAn,G,G,.,NM_198317.3,TGGAGCCTGGGAAAGCGTGGCGCCCATGAATATCCGCAGGT
2,chr1,1013982,1013983,G>A,1,+,1013982,1013983,"210,0,0",1,1,0,1028857|NM_005101.4(ISG15):c.4-1G>A,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,splice acceptor variant,nAn,nAn,RCV001330014,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""-""], [""g.1013983G>A"", ""-""]...",nAn,1,"Feb 21,2020",nAn,nAn,"NM_005101.4(ISG15):c.4-1G>A, Type: single nucl...",P,UNK,SUBST,1,1,1028857,nAn,1,1013983.0,1028857.0,G,A,.,.,"ALLELEID=1015460;CLNDISDB=MONDO:MONDO:0014502,...",chr1,Pathogenic,nAn,nAn,A,G,.,NM_005101.4,GAACCTGTGTGACGCCTGCAGGGCTGGGACCTGACGGTGAA
3,chr1,1014142,1014143,C>T,0,+,1014142,1014143,"210,0,0",1,1,0,183381|NM_005101.4(ISG15):c.163C>T (p.Gln55Ter),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,nonsense,786201005.0,nAn,RCV000162196,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""c.163C>T"", ""p.Gln55Ter""], [""g.18006C>T"", ""-...",nAn,1,"Jan 01,2015",nAn,"ClinGen:CA186084, <a target=_blank href='http:...","NM_005101.4(ISG15):c.163C>T (p.Gln55Ter), Type...",P,GERM,SUBST,1,0,183381,nAn,1,1014143.0,183381.0,C,T,.,.,"ALLELEID=181485;CLNDISDB=MONDO:MONDO:0014502,M...",chr1,Pathogenic,nAn,nAn,G,A,.,NM_005101.4,ACCCGAGCGGTGTGGCGCTGCAGGACAGGGTCCCCCTTGCC
4,chr1,1014227,1014228,G>A,1,+,1014227,1014228,"137,121,212",1,1,0,402986|NM_005101.4(ISG15):c.248G>A (p.Ser83Asn),Conflicting interpretations of pathogenicity,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,missense variant,1921.0,nAn,RCV000455759|RCV001519553|RCV001727727,N,not specified|Immunodeficiency 38 with basal g...,"MedGen:CN169374|MONDO:MONDO:0014502, <a target...","de novo,germline",GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""p.Ser83Asn""], [""c.248G>A"",...",nAn,4,"Sep 22,2021",nAn,ClinGen:CA507658,"NM_005101.4(ISG15):c.248G>A (p.Ser83Asn), Type...",CF,GERM,SUBST,1,1,402986,nAn

In [119]:
# Distribution of the mutations according to name
clinvar_df.groupby(['name'])['name'].count()
# Checking if there is a nan value
clinvar_df[clinvar_df['name'].isna()]

,chrom,chromStart,chromEnd,name,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,dna_seq


In [120]:
# Checking if there is a nan value
clinvar_df[clinvar_df['origName'].isna()]
clinvar_df[clinvar_df['origName']=='nAn']
clinvar_df[clinvar_df['origName']=='Nan']
clinvar_df[clinvar_df['origName']=='Na']

,chrom,chromStart,chromEnd,name,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,dna_seq


In [121]:
# Checking if there is a nan value
clinvar_df[clinvar_df['clinSign'].isna()]
# Distribution of the mutations according to clinSign
#clinvar_df.groupby(['clinSign'])['clinSign'].count()
# Distribution of the mutations according to CLNSIG - There is the same distribution between the column from UCSC and VCF
clinvar_df.groupby(['CLNSIG'])['CLNSIG'].count()


CLNSIG
Conflicting_interpretations_of_pathogenicity                               7395
Conflicting_interpretations_of_pathogenicity,_Affects                         1
Conflicting_interpretations_of_pathogenicity,_association                     2
Conflicting_interpretations_of_pathogenicity,_association,_risk_factor        2
Conflicting_interpretations_of_pathogenicity,_drug_response                   4
Conflicting_interpretations_of_pathogenicity,_other                          14
Conflicting_interpretations_of_pathogenicity,_other,_risk_factor              2
Conflicting_interpretations_of_pathogenicity,_protective                      1
Conflicting_interpretations_of_pathogenicity,_risk_factor                     6
Likely_pathogenic                                                         31850
Likely_pathogenic,_Affects                                                    1
Likely_pathogenic,_drug_response                                              7
Likely_pathogenic,_other         

In [122]:
# Distribution of the mutations according to type
clinvar_df.groupby(['type'])['type'].count()

type
single nucleotide variant    98513
Name: type, dtype: int64

In [123]:
# Distribution of the mutations according to molConseq
clinvar_df.groupby(['molConseq'])['molConseq'].count()

molConseq
3 prime UTR variant                 50
5 prime UTR variant                117
initiatior codon variant           939
intron variant                    2441
missense variant                 47748
non-coding transcript variant      238
nonsense                         30237
splice acceptor variant           7224
splice donor variant              8803
stop lost                          155
synonymous variant                 561
Name: molConseq, dtype: int64

In [124]:
# Distribution of the mutations according to phenotypeList
clinvar_df[clinvar_df['phenotypeList']=='not provided']

,chrom,chromStart,chromEnd,name,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,dna_seq
0,chr1,943994,943995,C>T,1,+,943994,943995,"210,0,0",1,1,0,950448|NM_001385641.1(SAMD11):c.2377C>T (p.Arg...,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,148398|SAMD11,nonsense,nAn,nAn,RCV001222161,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.943995C>T"", ""-""], [""c.2380C>T"", ""p.Arg794...",nAn,1,"Jul 13,2020",nAn,nAn,NM_001385641.1(SAMD11):c.2377C>T (p.Arg793Ter)...,P,GERM,SUBST,1,1,950448,nAn,1,943995.0,950448.0,C,T,.,.,ALLELEID=929884;CLNDISDB=MedGen:CN517202;CLNDN...,chr1,Pathogenic,nAn,nAn,G,G,.,NM_001385641.1,CAACCCTGCGGGCCCCGGAGCGAGAACTCGGCACAGGAGAG
14,chr1,1049671,1049672,C>T,1,+,1049671,1049672,"210,0,0",1,1,0,489335|NM_198576.4(AGRN):c.4621C>T (p.Arg1541Ter),Likely pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,375790|AGRN,nonsense,750176911.0,nAn,RCV000578890,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.34550C>T"", ""-""], [""g.1049672C>T"", ""-""], [...",nAn,1,"Jan 05,2018",nAn,ClinGen:CA337778755,"NM_198576.4(AGRN):c.4621C>T (p.Arg1541Ter), Ty...",LP,GERM,SUBST,1,1,489335,nAn,1,1049672.0,489335.0,C,T,.,.,ALLELEID=481556;CLNDISDB=MedGen:CN517202;CLNDN...,chr1,Likely_pathogenic,nAn,nAn,C,G,.,NM_198576.4,TTGGGCCGGGGGCTGCCACCCGAGGCTCTGGCGTGGGCGAG
26,chr1,1232516,1232517,G>A,1,+,1232516,1232517,"210,0,0",1,1,0,452846|NM_080605.4(B3GALT6):c.239G>A (p.Trp80Ter),Likely pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,126792|B3GALT6,nonsense,1553151208.0,nAn,RCV000520848,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.1232517G>A"", ""-""], [""g.5269G>A"", ""-""], [""...",nAn,1,"Oct 16,2017",nAn,ClinGen:CA337823610,"NM_080605.4(B3GALT6):c.239G>A (p.Trp80Ter), Ty...",LP,GERM,SUBST,1,1,452846,nAn,1,1232517.0,452846.0,G,A,.,.,ALLELEID=442612;CLNDISDB=MedGen:CN517202;CLNDN...,chr1,Likely_pathogenic,nAn,nAn,T,G,.,NM_080605.4,CAGCGTGATCCGCAGCACGTGGCTTGCGCGGCGCGGGGCCC
35,chr1,1232958,1232959,C>G,1,+,1232958,1232959,"210,0,0",1,1,0,806030|NM_080605.4(B3GALT6):c.681C>G (p.Tyr227...,Likely pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,126792|B3GALT6,nonsense,1375562480.0,nAn,RCV000993879,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.1232959C>G"", ""-""], [""g.5711C>G"", ""-""], [""...",nAn,1,"Jul 01,2019",nAn,nAn,"NM_080605.4(B3GALT6):c.681C>G (p.Tyr227Ter), T...",LP,GERM,SUBST,1,1,806030,nAn,1,1232959.0,806030.0,C,G,.,.,ALLELEID=794404;CLNDISDB=MedGen:CN517202;CLNDN...,chr1,Likely_pathogenic,nAn,nAn,A,C,.,NM_080605.4,TCGGCCGACCTGGTGCACTACCTGCGCCTCAGCCGCGACTA
45,chr1,1517412,1517413,G>A,1,+,1517412,1517413,"210,0,0",1,1,0,986833|NM_001170535.3(ATAD3A):c.384+1G>A,Likely pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,55210|ATAD3A,splice donor variant,nAn,nAn,RCV001267961,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.1517413G>A"", ""-""], [""g.10271G>A"", ""-""], [...",nAn,1,"Oct 23,2020",nAn,nAn,"NM_001170535.3(ATAD3A):c.384+1G>A, Type: singl...",LP,GERM,SUBST,1,1,986833,nAn,1,1517413.0,986833.0,G,A,.,.,ALLELEID=974951;CLNDISDB=MedGen:CN517202;CLNDN...,chr1,Likely_pathogenic,nAn,nAn,C,T,.,NM_001170535.3,AGACCCGGCAGCACCAGGCCGTAAGAGCGCAAGAGGCCGCG
...,...,.

In [125]:
# Distribution of the mutations according to base_before
clinvar_df.groupby(['base_before'])['base_before'].count()
# visual test
clinvar_df[clinvar_df['base_before']=='C']

,chrom,chromStart,chromEnd,name,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,dna_seq
5,chr1,1014358,1014359,G>T,0,+,1014358,1014359,"210,0,0",1,1,0,161454|NM_005101.4(ISG15):c.379G>T (p.Glu127Ter),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,nonsense,672601312.0,nAn,RCV000148988,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""c.379G>T"", ""p.Glu127Ter""], [""g.18222G>T"", ""...",nAn,1,"Jan 01,2015",nAn,"ClinGen:CA174067, <a target=_blank href='http:...","NM_005101.4(ISG15):c.379G>T (p.Glu127Ter), Typ...",P,GERM,SUBST,1,0,161454,nAn,1,1014359.0,161454.0,G,T,.,.,AF_EXAC=0.00001;ALLELEID=171288;CLNDISDB=MONDO...,chr1,Pathogenic,nAn,nAn,C,A,.,NM_005101.4,ACCTGTTCTGGCTGACCTTCGAGGGGAAGCCCCTGGAGGAC
11,chr1,1044110,1044111,G>T,1,+,1044110,1044111,"210,0,0",1,1,0,1076092|NM_198576.4(AGRN):c.2002G>T (p.Glu668Ter),Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,375790|AGRN,nonsense,nAn,nAn,RCV001389873,N,"Myasthenic syndrome,congenital,8","MONDO:MONDO:0014052, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""-"", ""-""], [""g.1044111G>T"", ""-""], [""g.28989G...",nAn,1,"May 22,2020",nAn,nAn,"NM_198576.4(AGRN):c.2002G>T (p.Glu668Ter), Typ...",P,GERM,SUBST,1,1,1076092,nAn,1,1044111.0,1076092.0,G,T,.,.,"ALLELEID=1058371;CLNDISDB=MONDO:MONDO:0014052,...",chr1,Pathogenic,nAn,nAn,C,A,.,NM_198576.4,TGCTCCCCTTCCCCGCAGCCGAGTGCGGTTCCGGAGGCTCT
14,chr1,1049671,1049672,C>T,1,+,1049671,1049672,"210,0,0",1,1,0,489335|NM_198576.4(AGRN):c.4621C>T (p.Arg1541Ter),Likely pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,375790|AGRN,nonsense,750176911.0,nAn,RCV000578890,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.34550C>T"", ""-""], [""g.1049672C>T"", ""-""], [...",nAn,1,"Jan 05,2018",nAn,ClinGen:CA337778755,"NM_198576.4(AGRN):c.4621C>T (p.Arg1541Ter), Ty...",LP,GERM,SUBST,1,1,489335,nAn,1,1049672.0,489335.0,C,T,.,.,ALLELEID=481556;CLNDISDB=MedGen:CN517202;CLNDN...,chr1,Likely_pathogenic,nAn,nAn,C,G,.,NM_198576.4,TTGGGCCGGGGGCTGCCACCCGAGGCTCTGGCGTGGGCGAG
15,chr1,1050448,1050449,G>A,0,+,1050448,1050449,"210,0,0",1,1,0,1284257|NM_198576.4(AGRN):c.4999G>A (p.Val1667...,Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,375790|AGRN,missense variant,nAn,nAn,RCV001698934,N,"Myasthenic syndrome,congenital,8","MONDO:MONDO:0014052, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""-"", ""-""], [""g.1050449G>A"", ""-""], [""g.35327G...",nAn,1,"Sep 22,2021",nAn,nAn,"NM_198576.4(AGRN):c.4999G>A (p.Val1667Met), Ty...",P,GERM,SUBST,1,0,1284257,nAn,1,1050449.0,1284257.0,G,A,.,.,"ALLELEID=1274112;CLNDISDB=MONDO:MONDO:0014052,...",chr1,Pathogenic,nAn,nAn,C,T,.,NM_198576.4,AGAAGATGGCGCTGGAGGTCGTGTTCCTGGCACGAGGCCCC
16,chr1,1050472,1050473,G>A,0,+,1050472,1050473,"210,0,0",1,1,0,243039|NM_198576.4(AGRN):c.5023G>A (p.Gly1675Ser),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,375790|AGRN,missense variant,764160563.0,nAn,RCV000235025,N,Congenital myasthenic syndrome,"MONDO:MONDO:0018940, MeSH:D020294, <a target=_...",germline,GRCh38,1p36.33,"[[""c.5023G>A"", ""p.Gly1675Ser""], [""g.35351G>A"",...",nAn,1,"Jul 14,2016",nAn,ClinGen:CA509824,"NM_198576.4(AGRN):c.5023G>A (p.Gly1675Ser), Ty...",P,GERM,SUBST,1,0,243039,nAn,1,1050473.0,243039.0,G,A,.,.,AF_EXAC=0.00003;ALLELEID=2441

In [126]:
# Distribution of the mutations according to base_after
clinvar_df.groupby(['base_after'])['base_after'].count()
# visual test
clinvar_df[clinvar_df['base_after']=='T']

,chrom,chromStart,chromEnd,name,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,dna_seq
15,chr1,1050448,1050449,G>A,0,+,1050448,1050449,"210,0,0",1,1,0,1284257|NM_198576.4(AGRN):c.4999G>A (p.Val1667...,Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,375790|AGRN,missense variant,nAn,nAn,RCV001698934,N,"Myasthenic syndrome,congenital,8","MONDO:MONDO:0014052, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""-"", ""-""], [""g.1050449G>A"", ""-""], [""g.35327G...",nAn,1,"Sep 22,2021",nAn,nAn,"NM_198576.4(AGRN):c.4999G>A (p.Val1667Met), Ty...",P,GERM,SUBST,1,0,1284257,nAn,1,1050449.0,1284257.0,G,A,.,.,"ALLELEID=1274112;CLNDISDB=MONDO:MONDO:0014052,...",chr1,Pathogenic,nAn,nAn,C,T,.,NM_198576.4,AGAAGATGGCGCTGGAGGTCGTGTTCCTGGCACGAGGCCCC
18,chr1,1050762,1050763,G>T,0,+,1050762,1050763,"210,0,0",1,1,0,126555|NM_198576.4(AGRN):c.5179G>T (p.Val1727Phe),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,375790|AGRN,missense variant,587777298.0,nAn,RCV000114427|RCV000235038,N,"Myasthenic syndrome,congenital,8|Congenital my...","MONDO:MONDO:0014052, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""g.35641G>T"", ""-""], [""c.5179G>T"", ""p.Val1727...",nAn,2,"Jul 14,2016",nAn,"ClinGen:CA151196, <a target=_blank href='http:...","NM_198576.4(AGRN):c.5179G>T (p.Val1727Phe), Ty...",P,GERM,SUBST,1,0,126555,nAn,1,1050763.0,126555.0,G,T,.,.,"ALLELEID=132068;CLNDISDB=MONDO:MONDO:0014052,M...",chr1,Pathogenic,nAn,nAn,G,T,.,NM_198576.4,CCCTGGGAGCCTGGACCAGGGTCTCACTGGAGCGAAACGGC
21,chr1,1232278,1232279,A>G,1,+,1232278,1232279,"210,0,0",1,1,0,60484|NM_080605.4(B3GALT6):c.1A>G (p.Met1Val),Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,126792|B3GALT6,initiatior codon variant,786200938.0,nAn,RCV000054390|RCV001560169,N,Spondyloepimetaphyseal dysplasia with joint la...,"MONDO:MONDO:0010075, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""p.Met1Val""], [""c.1A>G"", ""-...",nAn,2,"Sep 13,2019",nAn,"ClinGen:CA144521, <a target=_blank href='http:...","NM_080605.4(B3GALT6):c.1A>G (p.Met1Val), Type:...",P,GERM,SUBST,1,1,60484,nAn,1,1232279.0,60484.0,A,G,.,.,"ALLELEID=75079;CLNDISDB=MONDO:MONDO:0010075,Me...",chr1,Pathogenic,nAn,nAn,C,T,.,NM_080605.4,GCCGGCCCGGCGCGGGCGCCATGAAGCTGCTGCGGCGGGCG
31,chr1,1232833,1232834,T>C,0,+,1232833,1232834,"210,0,0",1,1,0,522415|NM_080605.4(B3GALT6):c.556T>C (p.Phe186...,Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,126792|B3GALT6,missense variant,1553151294.0,nAn,RCV000625553|RCV001090032|RCV001090033,N,Spondyloepimetaphyseal dysplasia with joint la...,"MONDO:MONDO:0010075, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""p.Phe186Leu""], [""c.556T>C""...",nAn,2,"Mar 17,2021",nAn,"ClinGen:CA337826592, <a target=_blank href='ht...","NM_080605.4(B3GALT6):c.556T>C (p.Phe186Leu), T...",P,GERM,SUBST,1,0,522415,nAn,1,1232834.0,522415.0,T,C,.,.,"ALLELEID=513007;CLNDISDB=MONDO:MONDO:0010075,M...",chr1,Pathogenic,nAn,nAn,C,T,.,NM_080605.4,GCCGCCTCTACTGGGGCTTCTTCTCGGGCCGCGGCCGCGTC
45,chr1,1517412,1517413,G>A,1,+,1517412,1517413,"210,0,0",1,1,0,986833|NM_001170535.3(ATAD3A):c.384+1G>A,Likely pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,55210|ATAD3A,splice donor variant,nAn,nAn,RCV001267961,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.1517413G>A"", ""-""], [""g.10271

In [127]:
# Checking if there is a nan value
clinvar_df[clinvar_df['gene_id'].isna()]
# checking how much refseq id with NM_ there is
clinvar_df[clinvar_df['gene_id'].str.contains('NM_')]
clinvar_df[~clinvar_df['gene_id'].str.contains('NM_')]
#len(clinvar_df[~clinvar_df['gene_id'].str.contains('NM_')])
# check if there is row without gene_id - there isn't
#clinvar_df[clinvar_df['gene_id']=='nAn']
#clinvar_df[clinvar_df['gene_id']=='Nan']
#clinvar_df[clinvar_df['gene_id']=='Na']
#clinvar_df[clinvar_df['gene_id'].isna()]

,chrom,chromStart,chromEnd,name,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,dna_seq
4020,chr1,145919694,145919695,T>C,0,+,145919694,145919695,"210,0,0",1,1,0,694601|NC_000001.11:g.145919695T>C,Likely pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,nAn,non-coding transcript variant,61746197.0,nAn,RCV000856620,N,Radial aplasia-thrombocytopenia syndrome,"MONDO:MONDO:0010121, <a target=_blank href='ht...",germline,GRCh38,1q21.1,"[[""g.145919695T>C"", ""-""], [""n.683T>C"", ""-""], [...",nAn,1,"Sep 16,2019",nAn,nAn,"NC_000001.11:g.145919695T>C, Type: single nucl...",LP,GERM,SUBST,1,0,694601,nAn,1,145919695.0,694601.0,T,C,.,.,AF_EXAC=0.00411;AF_TGP=0.00140;ALLELEID=682674...,chr1,Likely_pathogenic,nAn,nAn,A,C,.,NC_000001.11:g.145919695T>C,CCTAGATGCCACCTGGAATATCACTGTTCAATGGCTGGCTG
30588,chr15,79209843,79209844,C>T,0,+,79209843,79209844,"210,0,0",1,1,0,31007|NR_029705.1(MIR184):n.57C>T,Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,406960|MIR184,non-coding transcript variant,nAn,nAn,RCV000023998,N,EDICT syndrome,"MONDO:MONDO:0013678, <a target=_blank href='ht...",germline,GRCh38,15q25.1,"[[""g.79209844C>T"", ""-""], [""g.5057C>T"", ""-""], [...",nAn,1,"Jan 25,2012",nAn,<a target=_blank href='http://www.omim.org/ent...,"NR_029705.1(MIR184):n.57C>T, Type: single nucl...",P,GERM,SUBST,1,0,31007,nAn,15,79209844.0,31007.0,C,T,.,.,"ALLELEID=39964;CLNDISDB=MONDO:MONDO:0013678,Me...",chr15,Pathogenic,nAn,nAn,A,G,.,NR_029705.1,TTGTGACTGTAAGTGTTGGACGGAGAACTGATAAGGGTAGG
33497,chr16,23629275,23629276,G>T,1,-,23629275,23629276,"210,0,0",1,1,0,126657|PALB2:c.2515-1G>T,Likely pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,79728|PALB2,splice acceptor variant,587776417.0,nAn,RCV000001312|RCV000133479|RCV001030317|RCV0011...,N,Pancreatic cancer 3|not provided|Familial canc...,"MONDO:MONDO:0013236, <a target=_blank href='ht...",germline,GRCh38,16p12.2,"[[""-"", ""-""], [""-"", ""-""], [""g.23629276C>A"", ""-""...",nAn,5,"Sep 25,2020",nAn,"ClinGen:CA114879, <a target=_blank href='http:...","PALB2:c.2515-1G>T, Type: single nucleotide var...",LP,GERM,SUBST,1,1,126657,nAn,16,23629276.0,126657.0,C,A,.,.,"ALLELEID=132167;CLNDISDB=MONDO:MONDO:0013236,M...",chr16,Likely_pathogenic,nAn,nAn,A,A,.,PALB2:c.2515-1G>T,ATGTGACTTTTGTTTTCACAGACTGAAACAGCAGAGCTTCC
52867,chr2,121530884,121530885,T>C,1,+,121530884,121530885,"210,0,0",1,1,0,873520|NR_023343.1(RNU4ATAC):n.6T>C,Likely pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,100151683|RNU4ATAC,non-coding transcript variant,nAn,nAn,RCV001095771,N,RNU4ATAC-related spliceosomopathies,nAn,unknown,GRCh38,2q14.2,"[[""c.196-560A>G"", ""-""], [""c.196-560A>G"", ""-""],...",nAn,1,"Jan 31,2020",nAn,nAn,"NR_023343.1(RNU4ATAC):n.6T>C, Type: single nuc...",LP,UNK,SUBST,1,1,873520,nAn,2,121530885.0,873520.0,T,C,.,.,ALLELEID=861615;CLNDISDB=.;CLNDN=RNU4ATAC-rela...,chr2,Likely_pathogenic,nAn,nAn,A,C,.,NR_023343.1,GGGACTTTCTATTATAACCATCCTTTTCTTGGGGTTGCGCT
52869,chr2,121530886,121530887,C>T,1,+,121530886,121530887,"137,121,212",1,1,0,218086|NR_023343.1(RNU4ATAC):n.8C>T,Conflicting interpretations of pathogenicity,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,100151683|RNU4ATAC,non-coding transcript variant,370715569.0,nAn,RCV000202309|RCV001323592,N,Roifman syndrome|not provided,"MONDO:MONDO:0014722, <a target=_blank href='ht...",germline,GRCh38,2q14.2,"[[""c.196-562G>A"", ""-""], [""c.196-562G>A""

In [128]:
# check if there is row without dna_seq - there isn't
clinvar_df[clinvar_df['dna_seq']=='nAn']
#clinvar_df[clinvar_df['dna_seq']=='Nan']
#clinvar_df[clinvar_df['dna_seq']=='Na']
#clinvar_df[clinvar_df['dna_seq'].isna()]

,chrom,chromStart,chromEnd,name,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,dna_seq


In [129]:
# Distribution of the mutations according to numSubmit
clinvar_df.groupby(['numSubmit'])['numSubmit'].count()
# # Checking if there is a nan value
clinvar_df[clinvar_df['numSubmit'].isna()]

,chrom,chromStart,chromEnd,name,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,dna_seq


Continue creating the repository data base

In [130]:
# extract the location of mutation on exon 
clinvar_df['location_on_exon'] = clinvar_df['origName'].str.split('>').str[0].str.split('c.').str[1].str[:-1]
clinvar_df['location_on_exon']

0        2377
1        1682
2         4-1
3         163
4         248
         ... 
98508      89
98509      53
98510      12
98511       4
98512     206
Name: location_on_exon, Length: 98513, dtype: object

check location on exon

In [131]:
# Count how many nan values are in location_on_exon column
len(clinvar_df[clinvar_df['location_on_exon'].isna()])
clinvar_df[clinvar_df['location_on_exon'].isna()]

,chrom,chromStart,chromEnd,name,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,dna_seq,location_on_exon
4020,chr1,145919694,145919695,T>C,0,+,145919694,145919695,"210,0,0",1,1,0,694601|NC_000001.11:g.145919695T>C,Likely pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,nAn,non-coding transcript variant,61746197.0,nAn,RCV000856620,N,Radial aplasia-thrombocytopenia syndrome,"MONDO:MONDO:0010121, <a target=_blank href='ht...",germline,GRCh38,1q21.1,"[[""g.145919695T>C"", ""-""], [""n.683T>C"", ""-""], [...",nAn,1,"Sep 16,2019",nAn,nAn,"NC_000001.11:g.145919695T>C, Type: single nucl...",LP,GERM,SUBST,1,0,694601,nAn,1,145919695.0,694601.0,T,C,.,.,AF_EXAC=0.00411;AF_TGP=0.00140;ALLELEID=682674...,chr1,Likely_pathogenic,nAn,nAn,A,C,.,NC_000001.11:g.145919695T>C,CCTAGATGCCACCTGGAATATCACTGTTCAATGGCTGGCTG,NaN
30588,chr15,79209843,79209844,C>T,0,+,79209843,79209844,"210,0,0",1,1,0,31007|NR_029705.1(MIR184):n.57C>T,Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,406960|MIR184,non-coding transcript variant,nAn,nAn,RCV000023998,N,EDICT syndrome,"MONDO:MONDO:0013678, <a target=_blank href='ht...",germline,GRCh38,15q25.1,"[[""g.79209844C>T"", ""-""], [""g.5057C>T"", ""-""], [...",nAn,1,"Jan 25,2012",nAn,<a target=_blank href='http://www.omim.org/ent...,"NR_029705.1(MIR184):n.57C>T, Type: single nucl...",P,GERM,SUBST,1,0,31007,nAn,15,79209844.0,31007.0,C,T,.,.,"ALLELEID=39964;CLNDISDB=MONDO:MONDO:0013678,Me...",chr15,Pathogenic,nAn,nAn,A,G,.,NR_029705.1,TTGTGACTGTAAGTGTTGGACGGAGAACTGATAAGGGTAGG,NaN
52867,chr2,121530884,121530885,T>C,1,+,121530884,121530885,"210,0,0",1,1,0,873520|NR_023343.1(RNU4ATAC):n.6T>C,Likely pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,100151683|RNU4ATAC,non-coding transcript variant,nAn,nAn,RCV001095771,N,RNU4ATAC-related spliceosomopathies,nAn,unknown,GRCh38,2q14.2,"[[""c.196-560A>G"", ""-""], [""c.196-560A>G"", ""-""],...",nAn,1,"Jan 31,2020",nAn,nAn,"NR_023343.1(RNU4ATAC):n.6T>C, Type: single nuc...",LP,UNK,SUBST,1,1,873520,nAn,2,121530885.0,873520.0,T,C,.,.,ALLELEID=861615;CLNDISDB=.;CLNDN=RNU4ATAC-rela...,chr2,Likely_pathogenic,nAn,nAn,A,C,.,NR_023343.1,GGGACTTTCTATTATAACCATCCTTTTCTTGGGGTTGCGCT,NaN
52869,chr2,121530886,121530887,C>T,1,+,121530886,121530887,"137,121,212",1,1,0,218086|NR_023343.1(RNU4ATAC):n.8C>T,Conflicting interpretations of pathogenicity,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,100151683|RNU4ATAC,non-coding transcript variant,370715569.0,nAn,RCV000202309|RCV001323592,N,Roifman syndrome|not provided,"MONDO:MONDO:0014722, <a target=_blank href='ht...",germline,GRCh38,2q14.2,"[[""c.196-562G>A"", ""-""], [""c.196-562G>A"", ""-""],...",nAn,3,"Oct 19,2020",nAn,"ClinGen:CA210440, <a target=_blank href='http:...","NR_023343.1(RNU4ATAC):n.8C>T, Type: single nuc...",CF,GERM,SUBST,1,1,218086,nAn,2,121530887.0,218086.0,C,T,.,.,AF_EXAC=0.00037;AF_TGP=0.00080;ALLELEID=214740...,chr2,Conflicting_interpretations_of_pathogenicity,"Likely_pathogenic(1),Uncertain_significance(1)",yes,C,T,.,NR_023343.1,GACTTTCTATTATAACCATCCTTTTCTTGGGGTTGCGCTAC,NaN
52870,chr2,121530891,121530892,C>T,2,+,121530891,121530892,"210,0,0",1,1,0,218083|NR_023343.1(RNU4ATAC):n.13C>T,Pathogenic,&#9733;&#9733;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,100151683|RNU4ATAC,intron variant,559979281.0,nAn,RCV000202315|RCV000788934,N,Roifman syndrome|not provided,"MONDO:MONDO:0014722, <a target=_blank href='ht...","germline,matern

In [132]:
# Count how many non-numeric values (including nan values) there are in a column
check_digit_df = clinvar_df[~clinvar_df['location_on_exon'].fillna('').str.isdigit()]
check_digit_df

,chrom,chromStart,chromEnd,name,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,dna_seq,location_on_exon
2,chr1,1013982,1013983,G>A,1,+,1013982,1013983,"210,0,0",1,1,0,1028857|NM_005101.4(ISG15):c.4-1G>A,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,splice acceptor variant,nAn,nAn,RCV001330014,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""-""], [""g.1013983G>A"", ""-""]...",nAn,1,"Feb 21,2020",nAn,nAn,"NM_005101.4(ISG15):c.4-1G>A, Type: single nucl...",P,UNK,SUBST,1,1,1028857,nAn,1,1013983.0,1028857.0,G,A,.,.,"ALLELEID=1015460;CLNDISDB=MONDO:MONDO:0014502,...",chr1,Pathogenic,nAn,nAn,A,G,.,NM_005101.4,GAACCTGTGTGACGCCTGCAGGGCTGGGACCTGACGGTGAA,4-1
10,chr1,1043196,1043197,G>C,0,+,1043196,1043197,"210,0,0",1,1,0,1284256|NM_198576.4(AGRN):c.1385-42G>C,Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,375790|AGRN,intron variant,nAn,nAn,RCV001698933,N,"Myasthenic syndrome,congenital,8","MONDO:MONDO:0014052, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""g.1043197G>C"", ""-""], [""g.28075G>C"", ""-""], [...",nAn,1,"Sep 22,2021",nAn,nAn,"NM_198576.4(AGRN):c.1385-42G>C, Type: single n...",P,GERM,SUBST,1,0,1284256,nAn,1,1043197.0,1284256.0,G,C,.,.,"ALLELEID=1274111;CLNDISDB=MONDO:MONDO:0014052,...",chr1,Pathogenic,nAn,nAn,A,G,.,NM_198576.4,GGGGCTTTGCCTGCAGCGGAGGGGGGGCTTGTGGGACCACT,1385-42
40,chr1,1336515,1336516,G>A,0,-,1336515,1336516,"210,0,0",1,1,0,981468|NM_001330311.2(DVL1):c.1715-1G>A,Likely pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,1855|DVL1,splice acceptor variant,nAn,nAn,RCV001353069,N,"Robinow syndrome,autosomal dominant 2","MONDO:MONDO:0014591, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""c.1640-1G>A"", ""-""], [""g.1336516C>T"", ""-""], ...",nAn,1,"Oct 22,2019",nAn,nAn,"NM_001330311.2(DVL1):c.1715-1G>A, Type: single...",LP,UNK,SUBST,1,0,981468,nAn,1,1336516.0,981468.0,C,T,.,.,"ALLELEID=969551;CLNDISDB=MONDO:MONDO:0014591,M...",chr1,Likely_pathogenic,nAn,nAn,A,G,.,NM_001330311.2,CCATCCTGTTCTCACTCCCAGGGAGCAAAAGCAGTGGGTCC,1715-1
45,chr1,1517412,1517413,G>A,1,+,1517412,1517413,"210,0,0",1,1,0,986833|NM_001170535.3(ATAD3A):c.384+1G>A,Likely pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,55210|ATAD3A,splice donor variant,nAn,nAn,RCV001267961,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.1517413G>A"", ""-""], [""g.10271G>A"", ""-""], [...",nAn,1,"Oct 23,2020",nAn,nAn,"NM_001170535.3(ATAD3A):c.384+1G>A, Type: singl...",LP,GERM,SUBST,1,1,986833,nAn,1,1517413.0,986833.0,G,A,.,.,ALLELEID=974951;CLNDISDB=MedGen:CN517202;CLNDN...,chr1,Likely_pathogenic,nAn,nAn,C,T,.,NM_001170535.3,AGACCCGGCAGCACCAGGCCGTAAGAGCGCAAGAGGCCGCG,384+1
48,chr1,1523837,1523838,G>T,1,+,1523837,1523838,"210,0,0",1,1,0,989289|NM_001170535.3(ATAD3A):c.964-1G>T,Likely pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,55210|ATAD3A,splice acceptor variant,nAn,nAn,RCV001270778,N,Harel-Yoon syndrome,"MONDO:MONDO:0014958, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""c.1108-1G>T"", ""-""], [""g.1523838G>T"", ""-""], ...",nAn,1,"May 21,2019",nAn,nAn,"NM_001170535.3(ATAD3A):c.964-1G>T, Type: singl...",LP,UNK,SUBST,1,1,989289,nAn,1,1523838.0,989289.0,G,T,.,.,"ALLELEID=977169;CLNDISDB=MONDO:MONDO:0014958,M...",chr1,Likely_patho

In [133]:
# The distribution of molConseq  where there is nan value in location_on_exon  
check_digit_df.groupby('molConseq')['molConseq'].count()

molConseq
3 prime UTR variant                50
5 prime UTR variant               117
initiatior codon variant            8
intron variant                   2441
missense variant                   73
non-coding transcript variant     165
nonsense                          150
splice acceptor variant          7215
splice donor variant             8794
stop lost                           3
synonymous variant                  9
Name: molConseq, dtype: int64

In [134]:
clinvar_df

,chrom,chromStart,chromEnd,name,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,dna_seq,location_on_exon
0,chr1,943994,943995,C>T,1,+,943994,943995,"210,0,0",1,1,0,950448|NM_001385641.1(SAMD11):c.2377C>T (p.Arg...,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,148398|SAMD11,nonsense,nAn,nAn,RCV001222161,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.943995C>T"", ""-""], [""c.2380C>T"", ""p.Arg794...",nAn,1,"Jul 13,2020",nAn,nAn,NM_001385641.1(SAMD11):c.2377C>T (p.Arg793Ter)...,P,GERM,SUBST,1,1,950448,nAn,1,943995.0,950448.0,C,T,.,.,ALLELEID=929884;CLNDISDB=MedGen:CN517202;CLNDN...,chr1,Pathogenic,nAn,nAn,G,G,.,NM_001385641.1,CAACCCTGCGGGCCCCGGAGCGAGAACTCGGCACAGGAGAG,2377
1,chr1,964511,964512,C>A,0,+,964511,964512,"210,0,0",1,1,0,916564|NM_198317.3(KLHL17):c.1682C>A (p.Ala561...,Likely pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,339451|KLHL17,missense variant,nAn,nAn,RCV001172295,N,Tracheoesophageal fistula,"Human Phenotype Ontology:HP:0002575, MONDO:MON...",de novo,GRCh38,1p36.33,"[[""g.964512C>A"", ""-""], [""g.8926C>A"", ""-""], [""c...",nAn,1,"Jul 01,2019",nAn,nAn,"NM_198317.3(KLHL17):c.1682C>A (p.Ala561Glu), T...",LP,GERM,SUBST,1,0,916564,nAn,1,964512.0,916564.0,C,A,.,.,ALLELEID=904889;CLNDISDB=Human_Phenotype_Ontol...,chr1,Likely_pathogenic,nAn,nAn,G,G,.,NM_198317.3,TGGAGCCTGGGAAAGCGTGGCGCCCATGAATATCCGCAGGT,1682
2,chr1,1013982,1013983,G>A,1,+,1013982,1013983,"210,0,0",1,1,0,1028857|NM_005101.4(ISG15):c.4-1G>A,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,splice acceptor variant,nAn,nAn,RCV001330014,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""-""], [""g.1013983G>A"", ""-""]...",nAn,1,"Feb 21,2020",nAn,nAn,"NM_005101.4(ISG15):c.4-1G>A, Type: single nucl...",P,UNK,SUBST,1,1,1028857,nAn,1,1013983.0,1028857.0,G,A,.,.,"ALLELEID=1015460;CLNDISDB=MONDO:MONDO:0014502,...",chr1,Pathogenic,nAn,nAn,A,G,.,NM_005101.4,GAACCTGTGTGACGCCTGCAGGGCTGGGACCTGACGGTGAA,4-1
3,chr1,1014142,1014143,C>T,0,+,1014142,1014143,"210,0,0",1,1,0,183381|NM_005101.4(ISG15):c.163C>T (p.Gln55Ter),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,nonsense,786201005.0,nAn,RCV000162196,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""c.163C>T"", ""p.Gln55Ter""], [""g.18006C>T"", ""-...",nAn,1,"Jan 01,2015",nAn,"ClinGen:CA186084, <a target=_blank href='http:...","NM_005101.4(ISG15):c.163C>T (p.Gln55Ter), Type...",P,GERM,SUBST,1,0,183381,nAn,1,1014143.0,183381.0,C,T,.,.,"ALLELEID=181485;CLNDISDB=MONDO:MONDO:0014502,M...",chr1,Pathogenic,nAn,nAn,G,A,.,NM_005101.4,ACCCGAGCGGTGTGGCGCTGCAGGACAGGGTCCCCCTTGCC,163
4,chr1,1014227,1014228,G>A,1,+,1014227,1014228,"137,121,212",1,1,0,402986|NM_005101.4(ISG15):c.248G>A (p.Ser83Asn),Conflicting interpretations of pathogenicity,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,missense variant,1921.0,nAn,RCV000455759|RCV001519553|RCV001727727,N,not specified|Immunodeficiency 38 with basal g...,"MedGen:CN169374|MONDO:MONDO:0014502, <a target...","de novo,germline",GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""p.Ser83Asn""], [""c.248G>A"",...",nAn,4,"Sep 22,2021",nAn,ClinGen:CA507658,"NM_005101.4(ISG15):c.248G>A (p.Ser83Asn), Ty

create loc_in_three_nuc column

In [135]:
# # calculte the location of muation on codon
# #the stiil true one:
# #clinvar_df["loc_in_three_nuc"] = np.where(clinvar_df["Strand"]=='-',(clinvar_df["CDS_bases_before_mutation"]-clinvar_df["loc_in_frame"])%3,(clinvar_df["CDS_bases_after_mutation"]-clinvar_df["loc_in_frame"])%3)

# clinvar_df.loc[clinvar_df['CDS_bases_after_mutation'] =='Nan', 'loc_in_three_nuc'] = np.nan
# clinvar_df.loc[(clinvar_df['CDS_bases_before_mutation'] !='Nan')&(clinvar_df["Strand"]=='-') , 'loc_in_three_nuc'] = (clinvar_df["CDS_bases_before_mutation"].astype(float)-clinvar_df["loc_in_frame"].astype(float))%3
# clinvar_df.loc[(clinvar_df['CDS_bases_after_mutation'] !='Nan')&(clinvar_df["Strand"]=='+') , 'loc_in_three_nuc'] = (clinvar_df["CDS_bases_after_mutation"].astype(float)-clinvar_df["loc_in_frame"].astype(float))%3

# # clinvar_df["loc_in_three_nuc"] = np.where(clinvar_df["CDS_bases_after_mutation"]=='Nan',(np.nan))
# # clinvar_df["loc_in_three_nuc"] = np.where((~clinvar_df["CDS_bases_after_mutation"]=='Nan')&(clinvar_df["Strand"]=='-'),(clinvar_df["CDS_bases_before_mutation"]-clinvar_df["loc_in_frame"])%3)
# # clinvar_df["loc_in_three_nuc"] = np.where((~clinvar_df["CDS_bases_after_mutation"]=='Nan')&(clinvar_df["Strand"]=='+'),(clinvar_df["CDS_bases_after_mutation"]-clinvar_df["loc_in_frame"])%3)
# clinvar_df
# #np.where(clinvar_df["Strand"]=='-',(clinvar_df["CDS_bases_before_mutation"]-clinvar_df["loc_in_frame"])%3,(clinvar_df["CDS_bases_after_mutation"]-clinvar_df["loc_in_frame"])%3)

add RNA seq

In [136]:
# from Bio import SeqIO
# # Open the file and parse the sequences
# with open("/private5/Projects/dadush/clinvar_base_editing/cds_fasta", 'r') as handle:
#     for record in SeqIO.parse(handle, 'fasta'):
#         # Print the description of the first sequence
#         if '_alt' in record.description:
#             print(record.description)
#         # Exit the loop to only get the description of the first sequence
#         #break

In [137]:
# read thr RNA seqs file
from Bio import SeqIO
input_file = "/private5/Projects/dadush/clinvar_base_editing/cds_fasta"
#input_file = "/private5/Projects/dadush/clinvar_base_editing/cds_utrs.fasta"
#fasta_sequences = SeqIO.parse(open(input_file),'fasta')
seq_dict = {rec.id.replace('hg38_ncbiRefSeqCurated_', '')+rec.description : rec.seq for rec in SeqIO.parse(open(input_file), "fasta")} #if ('v1_' not in rec.description) or ('_alt' not in rec.description) or ('_fix' not in rec.description)}
seq_dict

{"NM_001276352.2hg38_ncbiRefSeqCurated_NM_001276352.2 range=chr1:67093580-67127240 5'pad=0 3'pad=0 strand=- repeatMasking=none": Seq('ATGGCAGAAAAAATCCTAGAGAAGTTGGATGTCCTTGATAAGCAAGCAGAGATA...TAA'),
 "NM_001276351.2hg38_ncbiRefSeqCurated_NM_001276351.2 range=chr1:67093005-67127240 5'pad=0 3'pad=0 strand=- repeatMasking=none": Seq('ATGGCAGAAAAAATCCTAGAGAAGTTGGATGTCCTTGATAAGCAAGCAGAGATA...TAA'),
 "NM_001005337.3hg38_ncbiRefSeqCurated_NM_001005337.3 range=chr1:201283703-201328836 5'pad=0 3'pad=0 strand=+ repeatMasking=none": Seq('ATGAACCACTCGCCGCTCAAGACCGCCTTGGCGTACGAATGCTTCCAGGACCAG...TAA'),
 "NM_000299.4hg38_ncbiRefSeqCurated_NM_000299.4 range=chr1:201283703-201328836 5'pad=0 3'pad=0 strand=+ repeatMasking=none": Seq('ATGAACCACTCGCCGCTCAAGACCGCCTTGGCGTACGAATGCTTCCAGGACCAG...TAA'),
 "NM_001042682.2hg38_ncbiRefSeqCurated_NM_001042682.2 range=chr1:8355087-8364133 5'pad=0 3'pad=0 strand=- repeatMasking=none": Seq('ATGTTCAAACCCGTCAAGGAAGAGGATGATGGGCTCAGTGGGAAGCATAGCATG...TAA'),
 "NM_012102.4h

In [138]:
[key for key, value in seq_dict.items() if 'NM_001372044.2' in key]

["NM_001372044.2hg38_ncbiRefSeqCurated_NM_001372044.2 range=chr22:50674415-50731312 5'pad=0 3'pad=0 strand=+ repeatMasking=none",
 "NM_001372044.2hg38_ncbiRefSeqCurated_NM_001372044.2 range=chr22_KQ759762v1_fix:10508-67428 5'pad=0 3'pad=0 strand=+ repeatMasking=none"]

In [139]:
#compare_dict = seq_dict.copy()
for key in seq_dict.copy():
    if 'v1_' in key :
        seq_dict.pop(key)
    elif '_alt' in key:
        seq_dict.pop(key)
    elif '_fix' in key:
        seq_dict.pop(key)
    elif 'v2_' in key:
        seq_dict.pop(key)
    else:
        new_key = key.split('hg38')[0]
        seq_dict[new_key] = seq_dict.pop(key)
len(seq_dict)       

62690

In [140]:
[key for key, value in seq_dict.items() if 'NM_001372044.2' in key]

['NM_001372044.2']

In [141]:
seq_dict

{'NM_001276352.2': Seq('ATGGCAGAAAAAATCCTAGAGAAGTTGGATGTCCTTGATAAGCAAGCAGAGATA...TAA'),
 'NM_001276351.2': Seq('ATGGCAGAAAAAATCCTAGAGAAGTTGGATGTCCTTGATAAGCAAGCAGAGATA...TAA'),
 'NM_001005337.3': Seq('ATGAACCACTCGCCGCTCAAGACCGCCTTGGCGTACGAATGCTTCCAGGACCAG...TAA'),
 'NM_000299.4': Seq('ATGAACCACTCGCCGCTCAAGACCGCCTTGGCGTACGAATGCTTCCAGGACCAG...TAA'),
 'NM_001042682.2': Seq('ATGTTCAAACCCGTCAAGGAAGAGGATGATGGGCTCAGTGGGAAGCATAGCATG...TAA'),
 'NM_012102.4': Seq('ATGACAGCGGACAAAGACAAAGACAAAGACAAAGAGAAGGACCGGGACCGAGAC...TAA'),
 'NM_001042681.2': Seq('ATGACAGCGGACAAAGACAAAGACAAAGACAAAGAGAAGGACCGGGACCGAGAC...TAA'),
 'NM_001281956.2': Seq('ATGCCGCGCTCGCGGGGACGGGAGCTGGGGCGCTGCGGCTGCCCCGCGGGGAGG...TAG'),
 'NM_052896.5': Seq('ATGAGGCTCATAAGCATCGCCCCCGGCCCTAGATGGCAAGTTCAAAGCCACCAC...TAG'),
 'NM_001320285.2': Seq('ATGATCTTCCTACTGTGTATTATTGGCTACATTGTTTTAGGACTTGTGGCCTGG...TAG'),
 'NM_001320287.2': Seq('ATGATCTTCCTACTGTGTATTATTGGCTACATTGTTTTAGGACTTGTGGCCTGG...TAG'),
 'NM_001130058.2': Seq('ATGAATGACACAGAAAA

In [142]:
# def that add exon seq to df
import re
def get_exon_seq(loc_in_exon,refseq_id):
    if pd.isna(loc_in_exon):
        return np.nan
    elif not loc_in_exon.isdigit():
        return np.nan
    else:
        #location =  re.sub("[^0-9]", "",loc_in_exon)
        location  = int(loc_in_exon)
        #for fasta in fasta_sequences:
        #fasta.id = fasta.id.replace('hg38_ncbiRefSeqCurated_', '')
        if refseq_id in seq_dict.keys():
            seq = seq_dict[refseq_id]
            if (location+20 > len(seq)) and (location-21<0):
                seq = seq[0:len(seq)]
            elif location+20 > len(seq):
                seq = seq[location-21:len(seq)]
            elif location-21<0:
                seq = seq[0:location+20]
            else:
                seq = seq[location-21:location+20]
            return str(seq)
        else:
            # refseq_id in str(fasta.id):
            #     seq=fasta.seq[int(location)-21:int(location)+20] 
            #     return str(seq)
            return np.nan       
# more cleaning
clinvar_df['new_exon_seq'] = clinvar_df.apply(lambda x: get_exon_seq(x.location_on_exon, x.gene_id),axis=1)
clinvar_df

,chrom,chromStart,chromEnd,name,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,dna_seq,location_on_exon,new_exon_seq
0,chr1,943994,943995,C>T,1,+,943994,943995,"210,0,0",1,1,0,950448|NM_001385641.1(SAMD11):c.2377C>T (p.Arg...,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,148398|SAMD11,nonsense,nAn,nAn,RCV001222161,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.943995C>T"", ""-""], [""c.2380C>T"", ""p.Arg794...",nAn,1,"Jul 13,2020",nAn,nAn,NM_001385641.1(SAMD11):c.2377C>T (p.Arg793Ter)...,P,GERM,SUBST,1,1,950448,nAn,1,943995.0,950448.0,C,T,.,.,ALLELEID=929884;CLNDISDB=MedGen:CN517202;CLNDN...,chr1,Pathogenic,nAn,nAn,G,G,.,NM_001385641.1,CAACCCTGCGGGCCCCGGAGCGAGAACTCGGCACAGGAGAG,2377,CAACCCTGCGGGCCCCGGAGCGAGAACTCGGCACAGGAGAG
1,chr1,964511,964512,C>A,0,+,964511,964512,"210,0,0",1,1,0,916564|NM_198317.3(KLHL17):c.1682C>A (p.Ala561...,Likely pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,339451|KLHL17,missense variant,nAn,nAn,RCV001172295,N,Tracheoesophageal fistula,"Human Phenotype Ontology:HP:0002575, MONDO:MON...",de novo,GRCh38,1p36.33,"[[""g.964512C>A"", ""-""], [""g.8926C>A"", ""-""], [""c...",nAn,1,"Jul 01,2019",nAn,nAn,"NM_198317.3(KLHL17):c.1682C>A (p.Ala561Glu), T...",LP,GERM,SUBST,1,0,916564,nAn,1,964512.0,916564.0,C,A,.,.,ALLELEID=904889;CLNDISDB=Human_Phenotype_Ontol...,chr1,Likely_pathogenic,nAn,nAn,G,G,.,NM_198317.3,TGGAGCCTGGGAAAGCGTGGCGCCCATGAATATCCGCAGGT,1682,TGGAGCCTGGGAAAGCGTGGCGCCCATGAATATCCGCAGGA
2,chr1,1013982,1013983,G>A,1,+,1013982,1013983,"210,0,0",1,1,0,1028857|NM_005101.4(ISG15):c.4-1G>A,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,splice acceptor variant,nAn,nAn,RCV001330014,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""-""], [""g.1013983G>A"", ""-""]...",nAn,1,"Feb 21,2020",nAn,nAn,"NM_005101.4(ISG15):c.4-1G>A, Type: single nucl...",P,UNK,SUBST,1,1,1028857,nAn,1,1013983.0,1028857.0,G,A,.,.,"ALLELEID=1015460;CLNDISDB=MONDO:MONDO:0014502,...",chr1,Pathogenic,nAn,nAn,A,G,.,NM_005101.4,GAACCTGTGTGACGCCTGCAGGGCTGGGACCTGACGGTGAA,4-1,NaN
3,chr1,1014142,1014143,C>T,0,+,1014142,1014143,"210,0,0",1,1,0,183381|NM_005101.4(ISG15):c.163C>T (p.Gln55Ter),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,nonsense,786201005.0,nAn,RCV000162196,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""c.163C>T"", ""p.Gln55Ter""], [""g.18006C>T"", ""-...",nAn,1,"Jan 01,2015",nAn,"ClinGen:CA186084, <a target=_blank href='http:...","NM_005101.4(ISG15):c.163C>T (p.Gln55Ter), Type...",P,GERM,SUBST,1,0,183381,nAn,1,1014143.0,183381.0,C,T,.,.,"ALLELEID=181485;CLNDISDB=MONDO:MONDO:0014502,M...",chr1,Pathogenic,nAn,nAn,G,A,.,NM_005101.4,ACCCGAGCGGTGTGGCGCTGCAGGACAGGGTCCCCCTTGCC,163,ACCCGAGCGGTGTGGCGCTGCAGGACAGGGTCCCCCTTGCC
4,chr1,1014227,1014228,G>A,1,+,1014227,1014228,"137,121,212",1,1,0,402986|NM_005101.4(ISG15):c.248G>A (p.Ser83Asn),Conflicting interpretations of pathogenicity,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,missense variant,1921.0,nAn,RCV000455759|RCV001519553|RCV001727727,N,not specified|Immunodeficiency 38 with basal g...,"MedGen:CN169374|MONDO:MONDO:0014502, <a target...","de novo,germline",GRCh38,1p36.33,"[[""

A new way to find the position in codon, using the location in the exon taken from the name of the mutation at UCSC

In [143]:

def get_loc_in_codon( loc_on_exon):
     if not str(loc_on_exon).isdigit():
          return np.nan
     else:
          result = int(loc_on_exon)%3
          if result==0:
               return 2
          elif result ==1:
               return 0
          elif result == 2:
               return 1
#apply the funcation get_change_in_amino_acid so that I get the mutant three nuc and assign it in new_alt_three_nuc column
clinvar_df['loc_in_nuc'] = clinvar_df.apply(lambda x: get_loc_in_codon(x.location_on_exon), axis=1)
clinvar_df

,chrom,chromStart,chromEnd,name,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,dna_seq,location_on_exon,new_exon_seq,loc_in_nuc
0,chr1,943994,943995,C>T,1,+,943994,943995,"210,0,0",1,1,0,950448|NM_001385641.1(SAMD11):c.2377C>T (p.Arg...,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,148398|SAMD11,nonsense,nAn,nAn,RCV001222161,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.943995C>T"", ""-""], [""c.2380C>T"", ""p.Arg794...",nAn,1,"Jul 13,2020",nAn,nAn,NM_001385641.1(SAMD11):c.2377C>T (p.Arg793Ter)...,P,GERM,SUBST,1,1,950448,nAn,1,943995.0,950448.0,C,T,.,.,ALLELEID=929884;CLNDISDB=MedGen:CN517202;CLNDN...,chr1,Pathogenic,nAn,nAn,G,G,.,NM_001385641.1,CAACCCTGCGGGCCCCGGAGCGAGAACTCGGCACAGGAGAG,2377,CAACCCTGCGGGCCCCGGAGCGAGAACTCGGCACAGGAGAG,0.0
1,chr1,964511,964512,C>A,0,+,964511,964512,"210,0,0",1,1,0,916564|NM_198317.3(KLHL17):c.1682C>A (p.Ala561...,Likely pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,339451|KLHL17,missense variant,nAn,nAn,RCV001172295,N,Tracheoesophageal fistula,"Human Phenotype Ontology:HP:0002575, MONDO:MON...",de novo,GRCh38,1p36.33,"[[""g.964512C>A"", ""-""], [""g.8926C>A"", ""-""], [""c...",nAn,1,"Jul 01,2019",nAn,nAn,"NM_198317.3(KLHL17):c.1682C>A (p.Ala561Glu), T...",LP,GERM,SUBST,1,0,916564,nAn,1,964512.0,916564.0,C,A,.,.,ALLELEID=904889;CLNDISDB=Human_Phenotype_Ontol...,chr1,Likely_pathogenic,nAn,nAn,G,G,.,NM_198317.3,TGGAGCCTGGGAAAGCGTGGCGCCCATGAATATCCGCAGGT,1682,TGGAGCCTGGGAAAGCGTGGCGCCCATGAATATCCGCAGGA,1.0
2,chr1,1013982,1013983,G>A,1,+,1013982,1013983,"210,0,0",1,1,0,1028857|NM_005101.4(ISG15):c.4-1G>A,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,splice acceptor variant,nAn,nAn,RCV001330014,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""-""], [""g.1013983G>A"", ""-""]...",nAn,1,"Feb 21,2020",nAn,nAn,"NM_005101.4(ISG15):c.4-1G>A, Type: single nucl...",P,UNK,SUBST,1,1,1028857,nAn,1,1013983.0,1028857.0,G,A,.,.,"ALLELEID=1015460;CLNDISDB=MONDO:MONDO:0014502,...",chr1,Pathogenic,nAn,nAn,A,G,.,NM_005101.4,GAACCTGTGTGACGCCTGCAGGGCTGGGACCTGACGGTGAA,4-1,NaN,NaN
3,chr1,1014142,1014143,C>T,0,+,1014142,1014143,"210,0,0",1,1,0,183381|NM_005101.4(ISG15):c.163C>T (p.Gln55Ter),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,nonsense,786201005.0,nAn,RCV000162196,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""c.163C>T"", ""p.Gln55Ter""], [""g.18006C>T"", ""-...",nAn,1,"Jan 01,2015",nAn,"ClinGen:CA186084, <a target=_blank href='http:...","NM_005101.4(ISG15):c.163C>T (p.Gln55Ter), Type...",P,GERM,SUBST,1,0,183381,nAn,1,1014143.0,183381.0,C,T,.,.,"ALLELEID=181485;CLNDISDB=MONDO:MONDO:0014502,M...",chr1,Pathogenic,nAn,nAn,G,A,.,NM_005101.4,ACCCGAGCGGTGTGGCGCTGCAGGACAGGGTCCCCCTTGCC,163,ACCCGAGCGGTGTGGCGCTGCAGGACAGGGTCCCCCTTGCC,0.0
4,chr1,1014227,1014228,G>A,1,+,1014227,1014228,"137,121,212",1,1,0,402986|NM_005101.4(ISG15):c.248G>A (p.Ser83Asn),Conflicting interpretations of pathogenicity,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,missense variant,1921.0,nAn,RCV000455759|RCV001519553|RCV001727727,N,not specified|Immunodeficiency 38 with basal g...,"MedGen:CN169374|MONDO:MONDO:0014502, <a target...","de novo,ger

In [144]:
# def that add codon seq to df
import re
def get_codon(loc_in_exon,refseq_id,loc_in_three):
    if pd.isna(loc_in_exon)or pd.isna(loc_in_three) :
        return np.nan
    else:
        location  = int(loc_in_exon)
        #for fasta in fasta_sequences:
        #fasta.id = fasta.id.replace('hg38_ncbiRefSeqCurated_', '')
        if refseq_id in seq_dict.keys():
            seq = seq_dict[refseq_id]
            if int(loc_in_three)==0:
                seq = seq[location-1:location+2]
            elif int(loc_in_three)==1:
                seq = seq[location-2:location+1]
            elif int(loc_in_three)==2:
                seq = seq[location-3:location]
            return str(seq)
        else:
            # refseq_id in str(fasta.id):
            #     seq=fasta.seq[int(location)-21:int(location)+20] 
            #     return str(seq)
            return np.nan       
# more cleaning
clinvar_df['new_codon_seq'] = clinvar_df.apply(lambda x: get_codon(x.location_on_exon, x.gene_id,x.loc_in_nuc),axis=1)#x.Strand
#selc_clinvar_df =clinvar_df[['chrom', 'chromStart','chromEnd','mutation','strand','loc_in_three_nuc','three_nuc','new_codon_seq']].copy()
#selc_clinvar_df.tail(n=50)
clinvar_df

,chrom,chromStart,chromEnd,name,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,dna_seq,location_on_exon,new_exon_seq,loc_in_nuc,new_codon_seq
0,chr1,943994,943995,C>T,1,+,943994,943995,"210,0,0",1,1,0,950448|NM_001385641.1(SAMD11):c.2377C>T (p.Arg...,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,148398|SAMD11,nonsense,nAn,nAn,RCV001222161,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.943995C>T"", ""-""], [""c.2380C>T"", ""p.Arg794...",nAn,1,"Jul 13,2020",nAn,nAn,NM_001385641.1(SAMD11):c.2377C>T (p.Arg793Ter)...,P,GERM,SUBST,1,1,950448,nAn,1,943995.0,950448.0,C,T,.,.,ALLELEID=929884;CLNDISDB=MedGen:CN517202;CLNDN...,chr1,Pathogenic,nAn,nAn,G,G,.,NM_001385641.1,CAACCCTGCGGGCCCCGGAGCGAGAACTCGGCACAGGAGAG,2377,CAACCCTGCGGGCCCCGGAGCGAGAACTCGGCACAGGAGAG,0.0,CGA
1,chr1,964511,964512,C>A,0,+,964511,964512,"210,0,0",1,1,0,916564|NM_198317.3(KLHL17):c.1682C>A (p.Ala561...,Likely pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,339451|KLHL17,missense variant,nAn,nAn,RCV001172295,N,Tracheoesophageal fistula,"Human Phenotype Ontology:HP:0002575, MONDO:MON...",de novo,GRCh38,1p36.33,"[[""g.964512C>A"", ""-""], [""g.8926C>A"", ""-""], [""c...",nAn,1,"Jul 01,2019",nAn,nAn,"NM_198317.3(KLHL17):c.1682C>A (p.Ala561Glu), T...",LP,GERM,SUBST,1,0,916564,nAn,1,964512.0,916564.0,C,A,.,.,ALLELEID=904889;CLNDISDB=Human_Phenotype_Ontol...,chr1,Likely_pathogenic,nAn,nAn,G,G,.,NM_198317.3,TGGAGCCTGGGAAAGCGTGGCGCCCATGAATATCCGCAGGT,1682,TGGAGCCTGGGAAAGCGTGGCGCCCATGAATATCCGCAGGA,1.0,GCG
2,chr1,1013982,1013983,G>A,1,+,1013982,1013983,"210,0,0",1,1,0,1028857|NM_005101.4(ISG15):c.4-1G>A,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,splice acceptor variant,nAn,nAn,RCV001330014,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""-""], [""g.1013983G>A"", ""-""]...",nAn,1,"Feb 21,2020",nAn,nAn,"NM_005101.4(ISG15):c.4-1G>A, Type: single nucl...",P,UNK,SUBST,1,1,1028857,nAn,1,1013983.0,1028857.0,G,A,.,.,"ALLELEID=1015460;CLNDISDB=MONDO:MONDO:0014502,...",chr1,Pathogenic,nAn,nAn,A,G,.,NM_005101.4,GAACCTGTGTGACGCCTGCAGGGCTGGGACCTGACGGTGAA,4-1,NaN,NaN,NaN
3,chr1,1014142,1014143,C>T,0,+,1014142,1014143,"210,0,0",1,1,0,183381|NM_005101.4(ISG15):c.163C>T (p.Gln55Ter),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,nonsense,786201005.0,nAn,RCV000162196,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""c.163C>T"", ""p.Gln55Ter""], [""g.18006C>T"", ""-...",nAn,1,"Jan 01,2015",nAn,"ClinGen:CA186084, <a target=_blank href='http:...","NM_005101.4(ISG15):c.163C>T (p.Gln55Ter), Type...",P,GERM,SUBST,1,0,183381,nAn,1,1014143.0,183381.0,C,T,.,.,"ALLELEID=181485;CLNDISDB=MONDO:MONDO:0014502,M...",chr1,Pathogenic,nAn,nAn,G,A,.,NM_005101.4,ACCCGAGCGGTGTGGCGCTGCAGGACAGGGTCCCCCTTGCC,163,ACCCGAGCGGTGTGGCGCTGCAGGACAGGGTCCCCCTTGCC,0.0,CAG
4,chr1,1014227,1014228,G>A,1,+,1014227,1014228,"137,121,212",1,1,0,402986|NM_005101.4(ISG15):c.248G>A (p.Ser83Asn),Conflicting interpretations of pathogenicity,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,missense variant,1921.0,nAn,RCV000455759|RCV001519553|RCV001727727,N,not specified|Immunodeficiency 38 with basal g...,"MedGen:CN169374|MONDO:MONDO:00145

check new_exon_seq column

In [145]:
#19590  rows has nan value in new_exon_seq because it location_on_exon is not number' for example: 4-1, 1385-42 and it include also the 50 rows above with nan value
clinvar_df[clinvar_df['new_exon_seq'].isna()]
clinvar_df[(clinvar_df['new_exon_seq'].isna())&(clinvar_df['location_on_exon'].astype(str).str.isdigit())]

,chrom,chromStart,chromEnd,name,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,dna_seq,location_on_exon,new_exon_seq,loc_in_nuc,new_codon_seq
784,chr1,17028610,17028611,G>T,2,-,17028610,17028611,"210,0,0",1,1,0,187177|NM_003000.2(SDHB):c.412G>T (p.Asp138Tyr),Likely pathogenic,&#9733;&#9733;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,6390|SDHB,missense variant,786203529.0,nAn,RCV000166877|RCV001729423,N,Hereditary cancer-predisposing syndrome|Paraga...,"MONDO:MONDO:0015356, <a target=_blank href='ht...","germline,unknown",GRCh38,1p36.13,"[[""c.412G>T"", ""-""], [""c.412G>T"", ""p.Asp138Tyr""...",nAn,2,"Aug 10,2021","ACMG2013,ACMG2016",ClinGen:CA015837,"NM_003000.2(SDHB):c.412G>T (p.Asp138Tyr), Type...",LP,GERM,SUBST,1,2,187177,nAn,1,17028611.0,187177.0,C,A,.,.,"ALLELEID=181615;CLNDISDB=MONDO:MONDO:0007273,M...",chr1,Likely_pathogenic,nAn,nAn,G,A,.,NM_003000.2,CACACATGTATGTGATAAAGGATCTTGTTCCCGTGAGTTTC,412,NaN,0.0,NaN
1212,chr1,26058481,26058482,C>T,1,-,26058481,26058482,"137,121,212",1,1,0,222849|NM_032588.3(TRIM63):c.739C>T (p.Gln247Ter),Conflicting interpretations of pathogenicity,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,84676|TRIM63,nonsense,148395034.0,nAn,RCV000208240|RCV000623023|RCV000850351|RCV0008...,N,Primary familial hypertrophic cardiomyopathy|I...,"MONDO:MONDO:0024573, MeSH:D024741, <a target=_...","germline,unknown",GRCh38,1p36.11,"[[""g.26058482G>A"", ""-""], [""g.14153C>T"", ""-""], ...",nAn,5,"Jul 07,2021",nAn,ClinGen:CA078758,"NM_032588.3(TRIM63):c.739C>T (p.Gln247Ter), Ty...",CF,GERM,SUBST,1,1,222849,nAn,1,26058482.0,222849.0,G,A,.,.,AF_ESP=0.00046;AF_EXAC=0.00049;AF_TGP=0.00020;...,chr1,Conflicting_interpretations_of_pathogenicity,"Likely_benign(1),Pathogenic(1),Uncertain_signi...",yes,C,A,.,NM_032588.3,GCTTCATCGAGGCCCTCATCCAGCAGTACCAGGAGCAGCTG,739,NaN,0.0,NaN
1321,chr1,29200573,29200574,C>T,2,-,29200573,29200574,"210,0,0",1,1,0,374882|NM_016011.4(MECR):c.772C>T (p.Arg258Trp),Pathogenic,&#9733;&#9733;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,51102|MECR,missense variant,145192716.0,nAn,RCV000415607|RCV000755160,N,"Dystonia,childhood-onset,with optic atrophy an...","MONDO:MONDO:0015003, <a target=_blank href='ht...","germline,inherited,paternal,unknown",GRCh38,1p35.3,"[[""g.29200574G>A"", ""-""], [""g.35385C>T"", ""-""], ...",nAn,4,"Jun 13,2019",nAn,<a target=_blank href='http://www.omim.org/ent...,"NM_016011.4(MECR):c.772C>T (p.Arg258Trp), Type...",P,GERM,SUBST,1,2,374882,nAn,1,29200574.0,374882.0,G,A,.,.,AF_ESP=0.00023;AF_EXAC=0.00008;ALLELEID=361787...,chr1,Pathogenic,nAn,nAn,A,G,.,NM_016011.4,TCCAGGACATGCCCCAGCCACGGCTTGCTCTCAACTGTGTT,772,NaN,0.0,NaN
1459,chr1,40078572,40078573,C>T,2,-,40078572,40078573,"210,0,0",1,1,0,236407|NM_000310.3(PPT1):c.713C>T (p.Pro238Leu),Pathogenic/Likely pathogenic,&#9733;&#9733;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,5538|PPT1,missense variant,878853322.0,nAn,RCV000225555,N,Neuronal ceroid lipofuscinosis 1,"MONDO:MONDO:0009744, <a target=_blank href='ht...","germline,inherited",GRCh38,1p34.2,"[[""g.23898C>T"", ""-""], [""g.40078573G>A"", ""-""], ...",nAn,3,"Jun 26,2020",nAn,ClinGen:CA10581621,"NM_000310.3(PPT1):c.713C>T (p.Pro238Leu), Type...",LP,GERM,SUBST,1,2,236407,nAn,1,40078573.0,236407.0,G,A,.,.,"ALLELEID=237963;CLNDISDB=MONDO:MONDO:0009744,M...",chr1,Pathogenic/Likely_pathogenic,nAn,nAn,C,T,.,NM_000310.3,CAATGATTCCATTGTGGACCCTGTAGATTCGGAGGTGAGAC,713,NaN,1.0,NaN
1488,chr1,40091397,40091398,A>T,2,-,40091397,40091398,"210,0,0",1,

In [146]:
clinvar_df[clinvar_df['loc_in_nuc'].isna()]


,chrom,chromStart,chromEnd,name,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,dna_seq,location_on_exon,new_exon_seq,loc_in_nuc,new_codon_seq
2,chr1,1013982,1013983,G>A,1,+,1013982,1013983,"210,0,0",1,1,0,1028857|NM_005101.4(ISG15):c.4-1G>A,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,splice acceptor variant,nAn,nAn,RCV001330014,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""-""], [""g.1013983G>A"", ""-""]...",nAn,1,"Feb 21,2020",nAn,nAn,"NM_005101.4(ISG15):c.4-1G>A, Type: single nucl...",P,UNK,SUBST,1,1,1028857,nAn,1,1013983.0,1028857.0,G,A,.,.,"ALLELEID=1015460;CLNDISDB=MONDO:MONDO:0014502,...",chr1,Pathogenic,nAn,nAn,A,G,.,NM_005101.4,GAACCTGTGTGACGCCTGCAGGGCTGGGACCTGACGGTGAA,4-1,NaN,NaN,NaN
10,chr1,1043196,1043197,G>C,0,+,1043196,1043197,"210,0,0",1,1,0,1284256|NM_198576.4(AGRN):c.1385-42G>C,Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,375790|AGRN,intron variant,nAn,nAn,RCV001698933,N,"Myasthenic syndrome,congenital,8","MONDO:MONDO:0014052, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""g.1043197G>C"", ""-""], [""g.28075G>C"", ""-""], [...",nAn,1,"Sep 22,2021",nAn,nAn,"NM_198576.4(AGRN):c.1385-42G>C, Type: single n...",P,GERM,SUBST,1,0,1284256,nAn,1,1043197.0,1284256.0,G,C,.,.,"ALLELEID=1274111;CLNDISDB=MONDO:MONDO:0014052,...",chr1,Pathogenic,nAn,nAn,A,G,.,NM_198576.4,GGGGCTTTGCCTGCAGCGGAGGGGGGGCTTGTGGGACCACT,1385-42,NaN,NaN,NaN
40,chr1,1336515,1336516,G>A,0,-,1336515,1336516,"210,0,0",1,1,0,981468|NM_001330311.2(DVL1):c.1715-1G>A,Likely pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,1855|DVL1,splice acceptor variant,nAn,nAn,RCV001353069,N,"Robinow syndrome,autosomal dominant 2","MONDO:MONDO:0014591, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""c.1640-1G>A"", ""-""], [""g.1336516C>T"", ""-""], ...",nAn,1,"Oct 22,2019",nAn,nAn,"NM_001330311.2(DVL1):c.1715-1G>A, Type: single...",LP,UNK,SUBST,1,0,981468,nAn,1,1336516.0,981468.0,C,T,.,.,"ALLELEID=969551;CLNDISDB=MONDO:MONDO:0014591,M...",chr1,Likely_pathogenic,nAn,nAn,A,G,.,NM_001330311.2,CCATCCTGTTCTCACTCCCAGGGAGCAAAAGCAGTGGGTCC,1715-1,NaN,NaN,NaN
45,chr1,1517412,1517413,G>A,1,+,1517412,1517413,"210,0,0",1,1,0,986833|NM_001170535.3(ATAD3A):c.384+1G>A,Likely pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,55210|ATAD3A,splice donor variant,nAn,nAn,RCV001267961,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.1517413G>A"", ""-""], [""g.10271G>A"", ""-""], [...",nAn,1,"Oct 23,2020",nAn,nAn,"NM_001170535.3(ATAD3A):c.384+1G>A, Type: singl...",LP,GERM,SUBST,1,1,986833,nAn,1,1517413.0,986833.0,G,A,.,.,ALLELEID=974951;CLNDISDB=MedGen:CN517202;CLNDN...,chr1,Likely_pathogenic,nAn,nAn,C,T,.,NM_001170535.3,AGACCCGGCAGCACCAGGCCGTAAGAGCGCAAGAGGCCGCG,384+1,NaN,NaN,NaN
48,chr1,1523837,1523838,G>T,1,+,1523837,1523838,"210,0,0",1,1,0,989289|NM_001170535.3(ATAD3A):c.964-1G>T,Likely pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,55210|ATAD3A,splice acceptor variant,nAn,nAn,RCV001270778,N,Harel-Yoon syndrome,"MONDO:MONDO:0014958, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""c.1108-1G>T"", ""-""], [""g.1523838G>T"", ""-""], ...",nAn,1,"May 21,2019",nAn,nAn,"NM_001170535.3(ATAD3A):c.964-1G>T, Type: singl...",LP,UNK,SUBST,1,1,989289,nAn,1,1523838.0,

In [147]:
clinvar_df[clinvar_df['new_codon_seq'].isna()]

,chrom,chromStart,chromEnd,name,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,dna_seq,location_on_exon,new_exon_seq,loc_in_nuc,new_codon_seq
2,chr1,1013982,1013983,G>A,1,+,1013982,1013983,"210,0,0",1,1,0,1028857|NM_005101.4(ISG15):c.4-1G>A,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,splice acceptor variant,nAn,nAn,RCV001330014,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""-""], [""g.1013983G>A"", ""-""]...",nAn,1,"Feb 21,2020",nAn,nAn,"NM_005101.4(ISG15):c.4-1G>A, Type: single nucl...",P,UNK,SUBST,1,1,1028857,nAn,1,1013983.0,1028857.0,G,A,.,.,"ALLELEID=1015460;CLNDISDB=MONDO:MONDO:0014502,...",chr1,Pathogenic,nAn,nAn,A,G,.,NM_005101.4,GAACCTGTGTGACGCCTGCAGGGCTGGGACCTGACGGTGAA,4-1,NaN,NaN,NaN
10,chr1,1043196,1043197,G>C,0,+,1043196,1043197,"210,0,0",1,1,0,1284256|NM_198576.4(AGRN):c.1385-42G>C,Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,375790|AGRN,intron variant,nAn,nAn,RCV001698933,N,"Myasthenic syndrome,congenital,8","MONDO:MONDO:0014052, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""g.1043197G>C"", ""-""], [""g.28075G>C"", ""-""], [...",nAn,1,"Sep 22,2021",nAn,nAn,"NM_198576.4(AGRN):c.1385-42G>C, Type: single n...",P,GERM,SUBST,1,0,1284256,nAn,1,1043197.0,1284256.0,G,C,.,.,"ALLELEID=1274111;CLNDISDB=MONDO:MONDO:0014052,...",chr1,Pathogenic,nAn,nAn,A,G,.,NM_198576.4,GGGGCTTTGCCTGCAGCGGAGGGGGGGCTTGTGGGACCACT,1385-42,NaN,NaN,NaN
40,chr1,1336515,1336516,G>A,0,-,1336515,1336516,"210,0,0",1,1,0,981468|NM_001330311.2(DVL1):c.1715-1G>A,Likely pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,1855|DVL1,splice acceptor variant,nAn,nAn,RCV001353069,N,"Robinow syndrome,autosomal dominant 2","MONDO:MONDO:0014591, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""c.1640-1G>A"", ""-""], [""g.1336516C>T"", ""-""], ...",nAn,1,"Oct 22,2019",nAn,nAn,"NM_001330311.2(DVL1):c.1715-1G>A, Type: single...",LP,UNK,SUBST,1,0,981468,nAn,1,1336516.0,981468.0,C,T,.,.,"ALLELEID=969551;CLNDISDB=MONDO:MONDO:0014591,M...",chr1,Likely_pathogenic,nAn,nAn,A,G,.,NM_001330311.2,CCATCCTGTTCTCACTCCCAGGGAGCAAAAGCAGTGGGTCC,1715-1,NaN,NaN,NaN
45,chr1,1517412,1517413,G>A,1,+,1517412,1517413,"210,0,0",1,1,0,986833|NM_001170535.3(ATAD3A):c.384+1G>A,Likely pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,55210|ATAD3A,splice donor variant,nAn,nAn,RCV001267961,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.1517413G>A"", ""-""], [""g.10271G>A"", ""-""], [...",nAn,1,"Oct 23,2020",nAn,nAn,"NM_001170535.3(ATAD3A):c.384+1G>A, Type: singl...",LP,GERM,SUBST,1,1,986833,nAn,1,1517413.0,986833.0,G,A,.,.,ALLELEID=974951;CLNDISDB=MedGen:CN517202;CLNDN...,chr1,Likely_pathogenic,nAn,nAn,C,T,.,NM_001170535.3,AGACCCGGCAGCACCAGGCCGTAAGAGCGCAAGAGGCCGCG,384+1,NaN,NaN,NaN
48,chr1,1523837,1523838,G>T,1,+,1523837,1523838,"210,0,0",1,1,0,989289|NM_001170535.3(ATAD3A):c.964-1G>T,Likely pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,55210|ATAD3A,splice acceptor variant,nAn,nAn,RCV001270778,N,Harel-Yoon syndrome,"MONDO:MONDO:0014958, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""c.1108-1G>T"", ""-""], [""g.1523838G>T"", ""-""], ...",nAn,1,"May 21,2019",nAn,nAn,"NM_001170535.3(ATAD3A):c.964-1G>T, Type: singl...",LP,UNK,SUBST,1,1,989289,nAn,1,1523838.0,

In [148]:
# visual test
clinvar_df[clinvar_df['loc_in_nuc']==2]

,chrom,chromStart,chromEnd,name,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,dna_seq,location_on_exon,new_exon_seq,loc_in_nuc,new_codon_seq
9,chr1,1042052,1042053,C>G,1,+,1042052,1042053,"210,0,0",1,1,0,644955|NM_198576.4(AGRN):c.1275C>G (p.Tyr425Ter),Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,375790|AGRN,nonsense,1239736447.0,nAn,RCV000798959,N,"Myasthenic syndrome,congenital,8","MONDO:MONDO:0014052, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""c.1275C>G"", ""-""], [""g.1042053C>G"", ""-""], [""...",nAn,1,"Jan 02,2019",nAn,nAn,"NM_198576.4(AGRN):c.1275C>G (p.Tyr425Ter), Typ...",P,GERM,SUBST,1,1,644955,nAn,1,1042053.0,644955.0,C,G,.,.,"ALLELEID=626510;CLNDISDB=MONDO:MONDO:0014052,M...",chr1,Pathogenic,nAn,nAn,A,A,.,NM_198576.4,GTCACCTGTGACGGGGCCTACAGGCCCGTGTGTGCCCAGGA,1275,GTCACCTGTGACGGGGCCTACAGGCCCGTGTGTGCCCAGGA,2.0,TAC
32,chr1,1232895,1232896,C>G,0,+,1232895,1232896,"210,0,0",1,1,0,869985|NM_080605.4(B3GALT6):c.618C>G (p.Cys206...,Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,126792|B3GALT6,missense variant,nAn,nAn,RCV001089599,N,Al-Gazali syndrome,"MONDO:MONDO:0012282, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""g.1232896C>G"", ""-""], [""g.5648C>G"", ""-""], [""...",nAn,1,"Apr 27,2020",nAn,<a target=_blank href='http://www.omim.org/ent...,"NM_080605.4(B3GALT6):c.618C>G (p.Cys206Trp), T...",P,GERM,SUBST,1,0,869985,nAn,1,1232896.0,869985.0,C,G,.,.,"ALLELEID=858244;CLNDISDB=MONDO:MONDO:0012282,M...",chr1,Pathogenic,nAn,nAn,G,G,.,NM_080605.4,GAGGCCGCCTGGCAACTCTGCGACTACTACCTGCCCTACGC,618,GAGGCCGCCTGGCAACTCTGCGACTACTACCTGCCCTACGC,2.0,TGC
35,chr1,1232958,1232959,C>G,1,+,1232958,1232959,"210,0,0",1,1,0,806030|NM_080605.4(B3GALT6):c.681C>G (p.Tyr227...,Likely pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,126792|B3GALT6,nonsense,1375562480.0,nAn,RCV000993879,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.1232959C>G"", ""-""], [""g.5711C>G"", ""-""], [""...",nAn,1,"Jul 01,2019",nAn,nAn,"NM_080605.4(B3GALT6):c.681C>G (p.Tyr227Ter), T...",LP,GERM,SUBST,1,1,806030,nAn,1,1232959.0,806030.0,C,G,.,.,ALLELEID=794404;CLNDISDB=MedGen:CN517202;CLNDN...,chr1,Likely_pathogenic,nAn,nAn,A,C,.,NM_080605.4,TCGGCCGACCTGGTGCACTACCTGCGCCTCAGCCGCGACTA,681,TCGGCCGACCTGGTGCACTACCTGCGCCTCAGCCGCGACTA,2.0,TAC
53,chr1,1535391,1535392,C>G,0,-,1535391,1535392,"210,0,0",1,1,0,161193|NM_001114748.2(TMEM240):c.489C>G (p.Tyr...,Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,339453|TMEM240,nonsense,606231452.0,nAn,RCV000148345,N,Spinocerebellar ataxia type 21,"MONDO:MONDO:0011833, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""g.28250G>C"", ""-""], [""c.489C>G"", ""p.Tyr163Te...",nAn,1,"Oct 01,2014",nAn,"ClinGen:CA272845, <a target=_blank href='http:...",NM_001114748.2(TMEM240):c.489C>G (p.Tyr163Ter)...,P,GERM,SUBST,1,0,161193,nAn,1,1535392.0,161193.0,G,C,.,.,"ALLELEID=171040;CLNDISDB=MONDO:MONDO:0011833,M...",chr1,Pathogenic,nAn,nAn,A,C,.,NM_001114748.2,CACGTGAAGCAGAAACTCTACCACAATGGCCACCCCAGCCC,489,CACGTGAAGCAGAAACTCTACCACAATGGCCACCCCAGCCC,2.0,TAC
88,chr1,1806513,1806514,T>G,0,-,1806513,1806514,"210,0,0",1,1,0,224712|NM_002074.5(GNB1):c.228T>G (p.Asp76Glu),Pathogenic/Likely pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,2782|GNB1,missense variant,869312822.0,nAn,RCV000210277|RCV000755058,N,"6 conditions|Neurodevelopmenta

In [149]:
# # select all rows that has rna seq - 78923  rows
# #clinvar_df[~clinvar_df['new_exon_seq'].isna()]
# # 565 rows has a regular number, so we were supposed to get an actual rna sequence, but the gene_id given is a gene_id from an older version, for example instead of .2 .3
# clinvar_df[(clinvar_df['new_exon_seq'].isna())&(clinvar_df['location_on_exon'].str.isdigit())]

# # There are 21423 rows in which the RNA sequence is not Nan and is different from the DNA sequence (that is, the mutation sits in the region of an exon edge) - and indeed the sequence is correct and similar to what happens in reality
# #clinvar_df[(~clinvar_df['new_exon_seq'].isna())&(clinvar_df['new_exon_seq']!=clinvar_df['dna_seq'])]


In [150]:
# # 19733 rows that nan in new_codon_seq
# #clinvar_df[clinvar_df['new_codon_seq'].isna()]
# # Of these, 19,590 are also with garbage value in rna seq
# #clinvar_df[(clinvar_df['new_exon_seq'].isna())&clinvar_df['new_codon_seq'].isna()]
# part_df = clinvar_df[~(clinvar_df['new_exon_seq'].isna())&clinvar_df['new_codon_seq'].isna()]
# part_df

In [151]:
# part_df.groupby('molConseq')['molConseq'].count()

check new_codon_seq col

In [152]:
# # There are 19733 rows that have a nan value in new_codon_seq and this is due to the fact that there was no merge with vcf for those rows
# clinvar_df[(clinvar_df['new_codon_seq'].isna())]

In [153]:
# #clinvar_df[(clinvar_df['new_codon_seq'].isna())&(clinvar_df['location_on_exon'].fillna('').str.isdigit())&(clinvar_df['loc_in_three_nuc'].isna())]
# # All rows that have a nan value in new_codon_seq are also in transcript_id
# clinvar_df[(clinvar_df['gene_id']!='Nan')&(clinvar_df['new_codon_seq'].isna())]

In [154]:
# # All lines where there is a nan value in transcript_id and not in new_exon_seq are cases of stop lost
# clinvar_df[(clinvar_df['gene_id']=='Nan')&(~clinvar_df['new_exon_seq'].isna())]

Checking if the code we received is indeed correct - part added at 22.02.23

In [155]:
#Extracting the amino acid name from the mutation name
clinvar_df['aa_in_origin_name'] = clinvar_df['origName'].str.split('\(p.').str[1].str[:3]
clinvar_df

,chrom,chromStart,chromEnd,name,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,dna_seq,location_on_exon,new_exon_seq,loc_in_nuc,new_codon_seq,aa_in_origin_name
0,chr1,943994,943995,C>T,1,+,943994,943995,"210,0,0",1,1,0,950448|NM_001385641.1(SAMD11):c.2377C>T (p.Arg...,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,148398|SAMD11,nonsense,nAn,nAn,RCV001222161,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.943995C>T"", ""-""], [""c.2380C>T"", ""p.Arg794...",nAn,1,"Jul 13,2020",nAn,nAn,NM_001385641.1(SAMD11):c.2377C>T (p.Arg793Ter)...,P,GERM,SUBST,1,1,950448,nAn,1,943995.0,950448.0,C,T,.,.,ALLELEID=929884;CLNDISDB=MedGen:CN517202;CLNDN...,chr1,Pathogenic,nAn,nAn,G,G,.,NM_001385641.1,CAACCCTGCGGGCCCCGGAGCGAGAACTCGGCACAGGAGAG,2377,CAACCCTGCGGGCCCCGGAGCGAGAACTCGGCACAGGAGAG,0.0,CGA,Arg
1,chr1,964511,964512,C>A,0,+,964511,964512,"210,0,0",1,1,0,916564|NM_198317.3(KLHL17):c.1682C>A (p.Ala561...,Likely pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,339451|KLHL17,missense variant,nAn,nAn,RCV001172295,N,Tracheoesophageal fistula,"Human Phenotype Ontology:HP:0002575, MONDO:MON...",de novo,GRCh38,1p36.33,"[[""g.964512C>A"", ""-""], [""g.8926C>A"", ""-""], [""c...",nAn,1,"Jul 01,2019",nAn,nAn,"NM_198317.3(KLHL17):c.1682C>A (p.Ala561Glu), T...",LP,GERM,SUBST,1,0,916564,nAn,1,964512.0,916564.0,C,A,.,.,ALLELEID=904889;CLNDISDB=Human_Phenotype_Ontol...,chr1,Likely_pathogenic,nAn,nAn,G,G,.,NM_198317.3,TGGAGCCTGGGAAAGCGTGGCGCCCATGAATATCCGCAGGT,1682,TGGAGCCTGGGAAAGCGTGGCGCCCATGAATATCCGCAGGA,1.0,GCG,Ala
2,chr1,1013982,1013983,G>A,1,+,1013982,1013983,"210,0,0",1,1,0,1028857|NM_005101.4(ISG15):c.4-1G>A,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,splice acceptor variant,nAn,nAn,RCV001330014,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""-""], [""g.1013983G>A"", ""-""]...",nAn,1,"Feb 21,2020",nAn,nAn,"NM_005101.4(ISG15):c.4-1G>A, Type: single nucl...",P,UNK,SUBST,1,1,1028857,nAn,1,1013983.0,1028857.0,G,A,.,.,"ALLELEID=1015460;CLNDISDB=MONDO:MONDO:0014502,...",chr1,Pathogenic,nAn,nAn,A,G,.,NM_005101.4,GAACCTGTGTGACGCCTGCAGGGCTGGGACCTGACGGTGAA,4-1,NaN,NaN,NaN,NaN
3,chr1,1014142,1014143,C>T,0,+,1014142,1014143,"210,0,0",1,1,0,183381|NM_005101.4(ISG15):c.163C>T (p.Gln55Ter),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,nonsense,786201005.0,nAn,RCV000162196,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""c.163C>T"", ""p.Gln55Ter""], [""g.18006C>T"", ""-...",nAn,1,"Jan 01,2015",nAn,"ClinGen:CA186084, <a target=_blank href='http:...","NM_005101.4(ISG15):c.163C>T (p.Gln55Ter), Type...",P,GERM,SUBST,1,0,183381,nAn,1,1014143.0,183381.0,C,T,.,.,"ALLELEID=181485;CLNDISDB=MONDO:MONDO:0014502,M...",chr1,Pathogenic,nAn,nAn,G,A,.,NM_005101.4,ACCCGAGCGGTGTGGCGCTGCAGGACAGGGTCCCCCTTGCC,163,ACCCGAGCGGTGTGGCGCTGCAGGACAGGGTCCCCCTTGCC,0.0,CAG,Gln
4,chr1,1014227,1014228,G>A,1,+,1014227,1014228,"137,121,212",1,1,0,402986|NM_005101.4(ISG15):c.248G>A (p.Ser83Asn),Conflicting interpretations of pathogenicity,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,missense variant,1921.0,nAn,RCV000455759|RCV001519553|RCV001727727,N,not specified|Immunodeficiency 38 with basal g...,

In [156]:
from Bio.Seq import Seq
from Bio.SeqUtils import seq3
# A function that accepts the codon and returns the name of the amino acid in a three letter format
def get_amino_acid(codon):
    # Checking whether this is an instance of a string
    if isinstance(codon, str):
        tran_cod = Seq(codon).translate()
        seq_three =  seq3(tran_cod)
        return seq_three
    else:
        return np.nan

clinvar_df['AA'] =clinvar_df.apply(lambda x: get_amino_acid(x.new_codon_seq), axis=1)

In [157]:
# Checking how many lines the amino acid has from the original name and the amino acid obtained from the translation is the same
clinvar_df [clinvar_df['AA']==clinvar_df['aa_in_origin_name']]
clinvar_df [(clinvar_df['AA']==clinvar_df['aa_in_origin_name'])&(clinvar_df['aa_in_origin_name'].isna())]
clinvar_df[(clinvar_df['aa_in_origin_name'].isna())].groupby('molConseq')['molConseq'].count()
clinvar_df[(clinvar_df['AA'].isna())]
len(clinvar_df [(clinvar_df['AA']!=clinvar_df['aa_in_origin_name'])&(~clinvar_df['aa_in_origin_name'].isna())&(~clinvar_df['AA'].isna())])
clinvar_df [(clinvar_df['AA']!=clinvar_df['aa_in_origin_name'])&(~clinvar_df['aa_in_origin_name'].isna())&(~clinvar_df['AA'].isna())]

,chrom,chromStart,chromEnd,name,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,dna_seq,location_on_exon,new_exon_seq,loc_in_nuc,new_codon_seq,aa_in_origin_name,AA
21787,chr12,121627017,121627018,C>T,0,+,121627017,121627018,"210,0,0",1,1,0,1283|NM_032790.3(ORAI1):c.271C>T (p.Arg91Trp),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,84876|ORAI1,missense variant,118203993.0,nAn,RCV000001346,N,Combined immunodeficiency due to ORAI1 deficiency,"MONDO:MONDO:0013007, <a target=_blank href='ht...",germline,GRCh38,12q24.31,"[[""g.121627018C>T"", ""-""], [""g.5444C>T"", ""-""], ...",nAn,1,"Mar 21,2008",nAn,"ClinGen:CA114898, UniProtKB:Q96D31#VAR_026226,...","NM_032790.3(ORAI1):c.271C>T (p.Arg91Trp), Type...",P,GERM,SUBST,1,0,1283,nAn,12,121627018.0,1283.0,C,T,.,.,"ALLELEID=16322;CLNDISDB=MONDO:MONDO:0013007,Me...",chr12,Pathogenic,nAn,nAn,C,G,.,NM_032790.3,CCAAGCTTAAAGCCTCCAGCCGGACCTCGGCTCTGCTCTCC,271,GCGCCAAGCTTAAAGCCTCCAGCCGGACCTCGGCTCTGCTC,0.0,AGC,Arg,Ser
21788,chr12,121627036,121627037,C>G,0,+,121627036,121627037,"210,0,0",1,1,0,440860|NM_032790.3(ORAI1):c.290C>G (p.Ser97Cys),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,84876|ORAI1,missense variant,1555322610.0,nAn,RCV000509049,N,"Myopathy,tubular aggregate,2","MONDO:MONDO:0014383, <a target=_blank href='ht...",germline,GRCh38,12q24.31,"[[""g.121627037C>G"", ""-""], [""g.5463C>G"", ""-""], ...",nAn,1,"Oct 04,2017",nAn,"ClinGen:CA386981209, <a target=_blank href='ht...","NM_032790.3(ORAI1):c.290C>G (p.Ser97Cys), Type...",P,GERM,SUBST,1,0,440860,nAn,12,121627037.0,440860.0,C,G,.,.,"ALLELEID=434505;CLNDISDB=MONDO:MONDO:0014383,M...",chr12,Pathogenic,nAn,nAn,T,C,.,NM_032790.3,CCGGACCTCGGCTCTGCTCTCCGGCTTCGCCATGGTGAGCT,290,CAGCCGGACCTCGGCTCTGCTCTCCGGCTTCGCCATGGTGG,1.0,CTC,Ser,Leu
21789,chr12,121627038,121627039,G>A,1,+,121627038,121627039,"210,0,0",1,1,0,189256|NM_032790.3(ORAI1):c.292G>A (p.Gly98Ser),Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,84876|ORAI1,missense variant,786204796.0,nAn,RCV000169690|RCV001389996,N,"Myopathy,tubular aggregate,2|Combined immunode...","MONDO:MONDO:0014383, <a target=_blank href='ht...",germline,GRCh38,12q24.31,"[[""c.292G>A"", ""p.Gly98Ser""], [""g.5465G>A"", ""-""...",nAn,2,"Jun 09,2020",nAn,"ClinGen:CA199185, <a target=_blank href='http:...","NM_032790.3(ORAI1):c.292G>A (p.Gly98Ser), Type...",P,GERM,SUBST,1,1,189256,nAn,12,121627039.0,189256.0,G,A,.,.,"ALLELEID=187138;CLNDISDB=MONDO:MONDO:0013007,M...",chr12,Pathogenic,nAn,nAn,C,G,.,NM_032790.3,GGACCTCGGCTCTGCTCTCCGGCTTCGCCATGGTGAGCTCC,292,GCCGGACCTCGGCTCTGCTCTCCGGCTTCGCCATGGTGGCA,0.0,TCC,Gly,Ser
21790,chr12,121641044,121641045,C>A,0,+,121641044,121641045,"210,0,0",1,1,0,192287|NM_032790.3(ORAI1):c.308C>A (p.Ala103Glu),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,84876|ORAI1,missense variant,786205890.0,nAn,RCV000172859,N,Combined immunodeficiency due to ORAI1 deficiency,"MONDO:MONDO:0013007, <a target=_blank href='ht...",germline,GRCh38,12q24.31,"[[""g.121641045C>A"", ""-""], [""g.19471C>A"", ""-""],...",nAn,1,"Dec 01,2009",nAn,"ClinGen:CA200154, <a target=_blank href='http:...","NM_032790.3(ORAI1):c.308C>A (p.Ala103Glu), Typ...",P,GERM,SUBST,1,0,192287,nAn,12,121641045.0,192287.0,C,A,.,.,"ALLELEID=190080;CLNDISDB=MONDO:MONDO:0013007,M...",chr12,Pathogenic,nAn,nAn,G,A,.,NM_032790.3,GCCTGTCCCCCCGCAGGTGGCAATGGTGGAGGTGCAGCTGG,308,GCTCTCCGGCTTCGCCATGGTGGCAATGGTGGAGGTGCAGC,1.0,GTG,Ala,Val
21791

In [158]:
# clinvar_df[clinvar_df['origName']=='200867|NM_000218.3(KCNQ1):c.37A>T (p.Lys13Ter)']
# clinvar_df[clinvar_df['origName']=='5699|NM_182643.3(DLC1):c.2875A>G (p.Thr959Ala)']
# clinvar_df[clinvar_df['_variantId']==425569] 
# #clinvar_df[clinvar_df['_variantId']==200867] 
#clinvar_df[clinvar_df['_variantId']==375455] 

Testing why in mutations of the initiator codon variant it has a RNA sequence and a codon as opposed to a stop lost and the reason for this is that an initiation codon is included in the CDS as opposed to a stop codon

In [159]:
# init_df = clinvar_df[clinvar_df['molConseq']=='initiatior codon variant']
# init_df

In [160]:
# init_df[~(init_df['new_codon_seq'].isna())&~(init_df['new_exon_seq'].isna()) ]

In [161]:
# lost_df = clinvar_df[clinvar_df['molConseq']=='stop lost']
# lost_df

In [162]:
# # All four cases where stop lost mutations are associated with the VCF are cases where the mutation really is not stoplost but is according to a different isoform
# lost_df[~lost_df['new_codon_seq'].isna() ]

In [163]:
# check if it works by see exapmles on UCSC genome broswer
clinvar_df[(clinvar_df['Strand']=='-')&(clinvar_df['loc_in_nuc']==0)]

,chrom,chromStart,chromEnd,name,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,dna_seq,location_on_exon,new_exon_seq,loc_in_nuc,new_codon_seq,aa_in_origin_name,AA
20,chr1,1213737,1213738,C>T,0,-,1213737,1213738,"210,0,0",1,1,0,96692|NM_003327.4(TNFRSF4):c.193C>T (p.Arg65Cys),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,7293|TNFRSF4,missense variant,587777075.0,nAn,RCV000082860,N,Immunodeficiency 16,"MONDO:MONDO:0014268, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""c.193C>T"", ""p.Arg65Cys""], [""g.5901C>T"", ""-""...",nAn,1,"Aug 26,2013",nAn,"ClinGen:CA149686, UniProtKB:P43489#VAR_070942,...","NM_003327.4(TNFRSF4):c.193C>T (p.Arg65Cys), Ty...",P,GERM,SUBST,1,0,96692,nAn,1,1213738.0,96692.0,G,A,.,.,AF_EXAC=0.00003;ALLELEID=102585;CLNDISDB=MONDO...,chr1,Pathogenic,nAn,nAn,C,G,.,NM_003327.4,GCTCCCAGAACACGGTGTGCCGTCCGTGCGGGCCGGGCTTC,193,GCTCCCAGAACACGGTGTGCCGTCCGTGCGGGCCGGGCTTC,0.0,CGT,Arg,Arg
51,chr1,1535369,1535370,C>T,1,-,1535369,1535370,"210,0,0",1,1,0,161196|NM_001114748.2(TMEM240):c.511C>T (p.Arg...,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,339453|TMEM240,missense variant,606231455.0,nAn,RCV000148348|RCV001374896,N,Spinocerebellar ataxia type 21|Neurodevelopmen...,"MONDO:MONDO:0011833, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""g.28228G>A"", ""-""], [""c.511C>T"", ""-""], [""g.1...",nAn,2,"Apr 06,2020",nAn,"ClinGen:CA272849, UniProtKB:Q5SV17#VAR_071910,...",NM_001114748.2(TMEM240):c.511C>T (p.Arg171Trp)...,P,GERM,SUBST,1,1,161196,nAn,1,1535370.0,161196.0,G,A,.,.,"ALLELEID=171043;CLNDISDB=MONDO:MONDO:0011833,M...",chr1,Pathogenic,nAn,nAn,G,G,.,NM_001114748.2,ACAATGGCCACCCCAGCCCGCGGCACCTGTGAGCCGCACGG,511,ACAATGGCCACCCCAGCCCGCGGCACCTGTGA,0.0,CGG,Arg,Arg
55,chr1,1535615,1535616,C>T,0,-,1535615,1535616,"210,0,0",1,1,0,161194|NM_001114748.2(TMEM240):c.346C>T (p.Arg...,Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,339453|TMEM240,missense variant,606231453.0,nAn,RCV000148346,N,Spinocerebellar ataxia type 21,"MONDO:MONDO:0011833, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""g.28474G>A"", ""-""], [""c.346C>T"", ""-""], [""g.1...",nAn,1,"Oct 01,2014",nAn,"ClinGen:CA272847, UniProtKB:Q5SV17#VAR_071907,...",NM_001114748.2(TMEM240):c.346C>T (p.Arg116Cys)...,P,GERM,SUBST,1,0,161194,nAn,1,1535616.0,161194.0,G,A,.,.,"ALLELEID=171041;CLNDISDB=MONDO:MONDO:0011833,M...",chr1,Pathogenic,nAn,nAn,G,G,.,NM_001114748.2,GCGTCCTGCACTGCGCCGTGCGCGCCTGGAGAGCCGGACGG,346,GCGTCCTGCACTGCGCCGTGCGCGCCTGGAGAGCCGGACGG,0.0,CGC,Arg,Arg
57,chr1,1535765,1535766,G>A,1,-,1535765,1535766,"137,121,212",1,1,0,372833|NM_001114748.2(TMEM240):c.196G>A (p.Gly...,Conflicting interpretations of pathogenicity,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,339453|TMEM240,missense variant,1057518011.0,nAn,RCV000413958|RCV001267568|RCV001542542,N,not provided|Inborn genetic diseases|Spinocere...,"MedGen:CN517202|MeSH:D030342, MedGen:C0950123|...","de novo,germline",GRCh38,1p36.33,"[[""g.28624C>T"", ""-""], [""c.196G>A"", ""-""], [""g.1...",nAn,4,"Jan 04,2021",nAn,ClinGen:CA16042311,"NM_001114748.2(TMEM240):c.196G>A (p.Gly66Arg),...",CF,GERM,SUBST,1,1,372833,nAn,1,1535766.0,372833.0,C,T,.,.,"ALLELEID=359235;CLNDISDB=MONDO:MONDO:0011833,M...",chr1,Conflicting_interpretations_of_pathogenicity,"Likely_pathogenic(1),Pathogenic(1),Uncertain_s...",yes,C,G,.,NM_001114748.2,ACTACGTGATCCCGTACGACGGGGACCAGTCGGTGGTGGAC,196,ACTACGTGATCCCGTACGACGG

In [164]:
# clinvar_df = clinvar_df.fillna('ANA')
clinvar_df

,chrom,chromStart,chromEnd,name,score,Strand,thickStart,thickEnd,reserved,blockCount,blockSizes,chromStarts,origName,clinSign,reviewStatus,type,geneId,molConseq,snpId,nsvId,rcvAcc,testedInGtr,phenotypeList,phenotype,origin,assembly,cytogenetic,_jsonHgvsTable,_hgvsProt,numSubmit,lastEval,guidelines,otherIds,_mouseOver,_clinSignCode,_originCode,_allTypeCode,_varLen,_starCount,_variantId,_dbVarSsvId,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,new_chrom,CLNSIG,CLNSIGCONF,pathogenic_at_least_one_subbmiter,base_before,base_after,strand,gene_id,dna_seq,location_on_exon,new_exon_seq,loc_in_nuc,new_codon_seq,aa_in_origin_name,AA
0,chr1,943994,943995,C>T,1,+,943994,943995,"210,0,0",1,1,0,950448|NM_001385641.1(SAMD11):c.2377C>T (p.Arg...,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,148398|SAMD11,nonsense,nAn,nAn,RCV001222161,N,not provided,<a target=_blank href='https://www.ncbi.nlm.ni...,germline,GRCh38,1p36.33,"[[""g.943995C>T"", ""-""], [""c.2380C>T"", ""p.Arg794...",nAn,1,"Jul 13,2020",nAn,nAn,NM_001385641.1(SAMD11):c.2377C>T (p.Arg793Ter)...,P,GERM,SUBST,1,1,950448,nAn,1,943995.0,950448.0,C,T,.,.,ALLELEID=929884;CLNDISDB=MedGen:CN517202;CLNDN...,chr1,Pathogenic,nAn,nAn,G,G,.,NM_001385641.1,CAACCCTGCGGGCCCCGGAGCGAGAACTCGGCACAGGAGAG,2377,CAACCCTGCGGGCCCCGGAGCGAGAACTCGGCACAGGAGAG,0.0,CGA,Arg,Arg
1,chr1,964511,964512,C>A,0,+,964511,964512,"210,0,0",1,1,0,916564|NM_198317.3(KLHL17):c.1682C>A (p.Ala561...,Likely pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,339451|KLHL17,missense variant,nAn,nAn,RCV001172295,N,Tracheoesophageal fistula,"Human Phenotype Ontology:HP:0002575, MONDO:MON...",de novo,GRCh38,1p36.33,"[[""g.964512C>A"", ""-""], [""g.8926C>A"", ""-""], [""c...",nAn,1,"Jul 01,2019",nAn,nAn,"NM_198317.3(KLHL17):c.1682C>A (p.Ala561Glu), T...",LP,GERM,SUBST,1,0,916564,nAn,1,964512.0,916564.0,C,A,.,.,ALLELEID=904889;CLNDISDB=Human_Phenotype_Ontol...,chr1,Likely_pathogenic,nAn,nAn,G,G,.,NM_198317.3,TGGAGCCTGGGAAAGCGTGGCGCCCATGAATATCCGCAGGT,1682,TGGAGCCTGGGAAAGCGTGGCGCCCATGAATATCCGCAGGA,1.0,GCG,Ala,Ala
2,chr1,1013982,1013983,G>A,1,+,1013982,1013983,"210,0,0",1,1,0,1028857|NM_005101.4(ISG15):c.4-1G>A,Pathogenic,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,splice acceptor variant,nAn,nAn,RCV001330014,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",unknown,GRCh38,1p36.33,"[[""-"", ""-""], [""-"", ""-""], [""g.1013983G>A"", ""-""]...",nAn,1,"Feb 21,2020",nAn,nAn,"NM_005101.4(ISG15):c.4-1G>A, Type: single nucl...",P,UNK,SUBST,1,1,1028857,nAn,1,1013983.0,1028857.0,G,A,.,.,"ALLELEID=1015460;CLNDISDB=MONDO:MONDO:0014502,...",chr1,Pathogenic,nAn,nAn,A,G,.,NM_005101.4,GAACCTGTGTGACGCCTGCAGGGCTGGGACCTGACGGTGAA,4-1,NaN,NaN,NaN,NaN,NaN
3,chr1,1014142,1014143,C>T,0,+,1014142,1014143,"210,0,0",1,1,0,183381|NM_005101.4(ISG15):c.163C>T (p.Gln55Ter),Pathogenic,&#9734;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,nonsense,786201005.0,nAn,RCV000162196,N,Immunodeficiency 38 with basal ganglia calcifi...,"MONDO:MONDO:0014502, <a target=_blank href='ht...",germline,GRCh38,1p36.33,"[[""c.163C>T"", ""p.Gln55Ter""], [""g.18006C>T"", ""-...",nAn,1,"Jan 01,2015",nAn,"ClinGen:CA186084, <a target=_blank href='http:...","NM_005101.4(ISG15):c.163C>T (p.Gln55Ter), Type...",P,GERM,SUBST,1,0,183381,nAn,1,1014143.0,183381.0,C,T,.,.,"ALLELEID=181485;CLNDISDB=MONDO:MONDO:0014502,M...",chr1,Pathogenic,nAn,nAn,G,A,.,NM_005101.4,ACCCGAGCGGTGTGGCGCTGCAGGACAGGGTCCCCCTTGCC,163,ACCCGAGCGGTGTGGCGCTGCAGGACAGGGTCCCCCTTGCC,0.0,CAG,Gln,Gln
4,chr1,1014227,1014228,G>A,1,+,1014227,1014228,"137,121,212",1,1,0,402986|NM_005101.4(ISG15):c.248G>A (p.Ser83Asn),Conflicting interpretations of pathogenicity,&#9733;&#9734;&#9734;&#9734;&nbsp;&nbsp;<small...,single nucleotide variant,9636|ISG15,missense variant,1921.0,nAn,RCV000455759|RCV001519553|RCV001727727,N,not specified|Immunodeficiency 

In [110]:
# save df
clinvar_df.to_csv("/private5/Projects/dadush/clinvar_base_editing/Erez_15_03_23/Erez_without_merged_ver_1_with_DNAֹ_RNA_seq.tsv", index=False,sep="\t")